In [6]:
import streamlit as st
import pandas as pd
import requests
import numpy as np
import yfinance as yf
from dune_client.client import DuneClient

In [7]:
@st.cache_data()
def fetch_data_from_api(api_url, params=None):
    response = requests.get(api_url, params=params)
    if response.status_code == 200:
        data = response.json()
        if 'rows' in data['result']:
            return pd.DataFrame(data['result']['rows'])
        return data
    else:
        print(f"Failed to retrieve data: {response.status_code}")
        return pd.DataFrame()  # or an empty dict

2024-04-01 19:16:08.729 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager


In [8]:
def fetch_historical_data(api_url, api_key):
    # Use the API key either as a query parameter or in the headers
    params = {'vs_currency': 'usd', 'days': 'max', 'interval': 'daily', 'x_cg_demo_api_key': api_key}
    headers = {'x-cg-demo-api-key': api_key}  # Alternatively, use this header

    response = requests.get(api_url, params=params, headers=headers)

    if response.status_code == 200:
        # Parse the JSON response
        historical_pricedata = response.json()
        # Extract the 'prices' and 'market_caps' data
        historical_price = historical_pricedata['prices']
        market_cap = pd.DataFrame(historical_pricedata['market_caps'], columns=['date', 'marketcap'])

        # Convert the 'timestamp' column from UNIX timestamps in milliseconds to datetime objects
        history = pd.DataFrame(historical_price, columns=['timestamp', 'price'])
        history['date'] = pd.to_datetime(history['timestamp'], unit='ms')
        history.set_index('date', inplace=True)
        history.drop(columns='timestamp', inplace=True)

        vol = pd.DataFrame(historical_pricedata['total_volumes'], columns=['date', 'volume'])
        vol['date'] = pd.to_datetime(vol['date'], unit='ms')
        vol.set_index('date', inplace=True)
        
        return history, market_cap, vol
    else:
        print(f"Failed to retrieve data: {response.status_code}")
        return pd.DataFrame(), pd.DataFrame(), pd.DataFrame()

In [9]:
api_key_dune = st.secrets["api_key"]
api_key_cg = st.secrets["api_key_cg"]
api_key_FRED = st.secrets["FRED_API_KEY"]

In [10]:
dune = DuneClient(api_key_dune)

In [11]:
def fetch_dune_data(num):
    result = dune.get_latest_result(num)
    return pd.DataFrame(result.result.rows)

In [12]:
pd.options.display.float_format = '{:,.2f}'.format

### First, lets get MakerDAO Financial Statements from https://dune.com/steakhouse/makerdao

Balance Sheet

In [13]:
# Balance Sheet
#bs_raw = dune.get_latest_result(2840463)

In [14]:
#bs_df = pd.DataFrame(bs_raw.result.rows)
#bs_df['period'] = pd.to_datetime(bs_df['period'])
#bs_df.set_index('period', inplace=True)
#bs_df.index = bs_df.index.normalize()
#bs_df = bs_df.sort_index()

In [15]:
#bs_df

In [16]:
bs_path = '../data/csv/bs.csv'

In [17]:
#bs_df.to_csv(bs_path)

In [18]:
bs_csv = pd.read_csv(bs_path, index_col='period', parse_dates=True)

In [19]:
#categorizing items as asset, liability, or equity
def categorize_item(item):
    if item in ['Crypto-Loans', 'Real-World Assets', 'Others assets', 'Stablecoins']:
        return 'Assets'
    elif item in ['DAI','DSR']:  # Assuming DAI represents a liability here; adjust according to your accounting rules
        return 'Liabilities'
    elif item == 'Equity':
        return 'Equity'
    else:
        return 'Other'  # For any item not explicitly categorized

# Assuming 'df' is your DataFrame
bs_csv['category'] = bs_csv['item'].apply(categorize_item)



In [20]:
bs_csv = bs_csv.iloc[::-1]

In [21]:
bs_csv

,balance,item,normalized,category
period,,,,
2024-03-21 00:00:00+00:00,"-67,156,270.08",Equity,-0.01,Equity
2024-03-21 00:00:00+00:00,"-3,223,114,058.66",DAI,-0.69,Liabilities
2024-03-21 00:00:00+00:00,"1,232,860,684.32",Stablecoins,0.26,Assets
2024-03-21 00:00:00+00:00,"-1,384,700,394.26",DSR,-0.30,Liabilities
2024-03-21 00:00:00+00:00,"2,358,355,394.97",Crypto-Loans,0.50,Assets
...,...,...,...,...
2020-07-01 00:00:00+00:00,NaN,Stablecoins,NaN,Assets
2020-07-01 00:00:00+00:00,"-111,909,502.58",DAI,-0.77,Liabilities
2020-07-01 00:00:00+00:00,"144,805,383.35",Crypto-Loans,1.00,Assets


In [22]:
pivoted_balance_sheet = bs_csv.pivot(columns='item', values='balance')

In [23]:
# Percent Changes in account balances 
pivoted_balance_sheet['Crypto-Loans_pct_chg'] = pivoted_balance_sheet['Crypto-Loans'].pct_change()
pivoted_balance_sheet['DAI_pct_chg'] = pivoted_balance_sheet['DAI'].pct_change()
pivoted_balance_sheet['DSR_pct_chg'] = pivoted_balance_sheet['DSR'].pct_change()
pivoted_balance_sheet['Equity_pct_chg'] = pivoted_balance_sheet['Equity'].pct_change()
pivoted_balance_sheet['Others_assets_pct_chg'] = pivoted_balance_sheet['Others assets'].pct_change()  # Assuming this is the correct column name
pivoted_balance_sheet['Real-World_Assets_pct_chg'] = pivoted_balance_sheet['Real-World Assets'].pct_change()
pivoted_balance_sheet['Stablecoins_pct_chg'] = pivoted_balance_sheet['Stablecoins'].pct_change()


In [24]:
# Rolling Averages, Standard Deviation
# Define the window size for rolling calculation
window_size = 7  # for example, a 7-day rolling window

# Calculate rolling averages
pivoted_balance_sheet['Crypto-Loans_rolling_avg'] = pivoted_balance_sheet['Crypto-Loans'].rolling(window=window_size).mean()
pivoted_balance_sheet['DAI_rolling_avg'] = pivoted_balance_sheet['DAI'].rolling(window=window_size).mean()
pivoted_balance_sheet['DSR_rolling_avg'] = pivoted_balance_sheet['DSR'].rolling(window=window_size).mean()
pivoted_balance_sheet['Equity_rolling_avg'] = pivoted_balance_sheet['Equity'].rolling(window=window_size).mean()
pivoted_balance_sheet['Others_assets_rolling_avg'] = pivoted_balance_sheet['Others assets'].rolling(window=window_size).mean()
pivoted_balance_sheet['Real-World_Assets_rolling_avg'] = pivoted_balance_sheet['Real-World Assets'].rolling(window=window_size).mean()
pivoted_balance_sheet['Stablecoins_rolling_avg'] = pivoted_balance_sheet['Stablecoins'].rolling(window=window_size).mean()

# Calculate volatility (standard deviation)
pivoted_balance_sheet['Crypto-Loans_volatility'] = pivoted_balance_sheet['Crypto-Loans_pct_chg'].rolling(window=window_size).std()
pivoted_balance_sheet['DAI_volatility'] = pivoted_balance_sheet['DAI_pct_chg'].rolling(window=window_size).std()
pivoted_balance_sheet['DSR_volatility'] = pivoted_balance_sheet['DSR_pct_chg'].rolling(window=window_size).std()
pivoted_balance_sheet['Equity_volatility'] = pivoted_balance_sheet['Equity_pct_chg'].rolling(window=window_size).std()
pivoted_balance_sheet['Others_assets_volatility'] = pivoted_balance_sheet['Others_assets_pct_chg'].rolling(window=window_size).std()
pivoted_balance_sheet['Real-World_Assets_volatility'] = pivoted_balance_sheet['Real-World_Assets_pct_chg'].rolling(window=window_size).std()
pivoted_balance_sheet['Stablecoins_volatility'] = pivoted_balance_sheet['Stablecoins_pct_chg'].rolling(window=window_size).std()



In [25]:
# Assuming you've already calculated percent changes (_pct_chg)
window_size = 30  # Adjust based on your analysis needs

# Calculate rolling averages and volatilities based on percent changes
for col in ['Crypto-Loans', 'DAI', 'DSR', 'Equity', 'Others_assets', 'Real-World_Assets', 'Stablecoins']:
    pct_chg_col = f'{col}_pct_chg'  # The column names for percent changes you've calculated
    pivoted_balance_sheet[f'{col}_rolling_avg_pct_chg'] = pivoted_balance_sheet[pct_chg_col].rolling(window=window_size).mean()
    pivoted_balance_sheet[f'{col}_volatility_pct_chg'] = pivoted_balance_sheet[pct_chg_col].rolling(window=window_size).std()


In [26]:
pivoted_balance_sheet.columns = [f'b_s_{col}' if col != 'period' else col for col in pivoted_balance_sheet.columns]

In [27]:
pivoted_balance_sheet.tail()

,b_s_Crypto-Loans,b_s_DAI,b_s_DSR,b_s_Equity,b_s_Others assets,b_s_Real-World Assets,b_s_Stablecoins,b_s_Crypto-Loans_pct_chg,b_s_DAI_pct_chg,b_s_DSR_pct_chg,...,b_s_DSR_rolling_avg_pct_chg,b_s_DSR_volatility_pct_chg,b_s_Equity_rolling_avg_pct_chg,b_s_Equity_volatility_pct_chg,b_s_Others_assets_rolling_avg_pct_chg,b_s_Others_assets_volatility_pct_chg,b_s_Real-World_Assets_rolling_avg_pct_chg,b_s_Real-World_Assets_volatility_pct_chg,b_s_Stablecoins_rolling_avg_pct_chg,b_s_Stablecoins_volatility_pct_chg
period,,,,,,,,,,,,,,,,,,,,,
2024-03-17 00:00:00+00:00,"2,567,732,340.42","-3,225,717,987.93","-1,405,341,920.91","-68,936,169.75","48,533.09","1,082,824,203.90","1,049,391,001.19",-0.02,-0.01,0.03,...,0.01,0.05,0.01,0.04,0.00,0.00,-0.02,0.03,0.03,0.17
2024-03-18 00:00:00+00:00,"2,487,739,660.28","-3,236,587,097.37","-1,440,743,920.84","-67,073,216.58","48,533.09","1,084,684,721.77","1,171,931,319.66",-0.03,0.00,0.03,...,0.01,0.05,0.01,0.04,0.00,0.00,-0.02,0.03,0.03,0.17
2024-03-19 00:00:00+00:00,"2,442,828,415.48","-3,237,504,446.80","-1,399,485,314.96","-66,776,806.16","48,533.09","1,084,527,447.54","1,176,362,171.82",-0.02,0.00,-0.03,...,0.01,0.05,0.01,0.04,0.00,0.00,-0.02,0.03,0.03,0.17
2024-03-20 00:00:00+00:00,"2,358,443,170.28","-3,225,358,948.44","-1,373,045,029.84","-67,160,889.57","48,533.09","1,083,706,110.61","1,223,367,053.87",-0.03,-0.00,-0.02,...,0.01,0.05,0.01,0.04,0.00,0.00,-0.02,0.03,0.03,0.17
2024-03-21 00:00:00+00:00,"2,358,355,394.97","-3,223,114,058.66","-1,384,700,394.26","-67,156,270.08","48,533.09","1,083,706,110.61","1,232,860,684.32",-0.00,-0.00,0.01,...,0.01,0.05,0.01,0.04,0.00,0.00,-0.02,0.03,0.03,0.17


In [28]:
pivoted_balance_sheet.shape[0]

1360

MONTHLY Income Statement/PnL (also includes more detailed balance sheet)

In [22]:
#is_df = fetch_dune_data(2641549) 

In [23]:
#is_df.head()

In [24]:
#is_df_wide = is_df.pivot_table(index='period', columns='item', values='value', aggfunc='sum').reset_index()
#is_df_wide = is_df_wide.iloc[::-1]

In [25]:
#is_df_wide.head()

In [29]:
is_path = '../data/csv/is.csv'

In [30]:
#is_df.to_csv(is_path)

In [31]:
is_csv = pd.read_csv(is_path, index_col='period', parse_dates=True)

In [32]:
is_csv.head()

,Unnamed: 0,expenses,item,lending_income,liquidation_income,month,net_income,trading_income,value,year
period,,,,,,,,,,
2024-03-01,0,"-3,857,879.46",1 - PnL,"11,054,226.90","4,623.88",3,"7,200,971.32",0.00,NaN,2024
2024-03-01,1,NaN,1.1 - Lending Revenues,NaN,NaN,3,NaN,NaN,"18,130,771.84",2024
2024-03-01,2,NaN,1.2 - Liquidations Revenues,NaN,NaN,3,NaN,NaN,"4,623.88",2024
2024-03-01,3,NaN,1.3 - Trading Revenues,NaN,NaN,3,NaN,NaN,0.00,2024
2024-03-01,4,NaN,1.4 - Lending Expenses,NaN,NaN,3,NaN,NaN,"-7,076,544.94",2024


In [33]:
pivoted_income_statement = is_csv.pivot_table(index='period', 
                            columns='item', 
                            values='value', 
                            aggfunc='sum').reset_index()

In [34]:
pivoted_income_statement['Total Revenues']= pivoted_income_statement[['1 - PnL', '1.1 - Lending Revenues', '1.2 - Liquidations Revenues', '1.3 - Trading Revenues']].sum(axis=1)
pivoted_income_statement['Total Expenses'] = pivoted_income_statement[['1.4 - Lending Expenses', '1.5 - Liquidations Expenses', '1.6 - Workforce Expenses']].sum(axis=1)
pivoted_income_statement['profit_margin'] = pivoted_income_statement['1.9 - Net Income'] / pivoted_income_statement['2.9 - Total Assets']
pivoted_income_statement['ROA'] = pivoted_income_statement['1.9 - Net Income'] / pivoted_income_statement['2.9 - Total Assets']
pivoted_income_statement['ROE'] = pivoted_income_statement['1.9 - Net Income'] / pivoted_income_statement['3.7 - Equity (Surplus Buffer)']
pivoted_income_statement['debt_to_equity'] = pivoted_income_statement['3.1 - Liabilities (DAI)'] / pivoted_income_statement['3.7 - Equity (Surplus Buffer)']
pivoted_income_statement['debt_ratio'] = pivoted_income_statement['3.1 - Liabilities (DAI)'] / pivoted_income_statement['2.9 - Total Assets'] 
pivoted_income_statement['cumulative_revenues'] = pivoted_income_statement['Total Revenues'].cumsum()
pivoted_income_statement['cumulative_expenses'] = pivoted_income_statement['Total Expenses'].cumsum()
pivoted_income_statement['cumulative_net_income'] = pivoted_income_statement['1.9 - Net Income'].cumsum()


In [35]:
pivoted_income_statement.tail()

item,period,1 - PnL,1.1 - Lending Revenues,1.2 - Liquidations Revenues,1.3 - Trading Revenues,1.4 - Lending Expenses,1.5 - Liquidations Expenses,1.6 - Workforce Expenses,1.9 - Net Income,2 - Assets,...,Total Revenues,Total Expenses,profit_margin,ROA,ROE,debt_to_equity,debt_ratio,cumulative_revenues,cumulative_expenses,cumulative_net_income
48,2023-11-01,0.00,"29,666,040.79",455.03,0.00,"-6,376,789.60",0.00,"-2,974,302.25","20,315,403.96",0.00,...,"29,666,495.81","-9,351,091.85",0.00,0.00,0.35,90.89,0.99,"291,977,550.62","-133,751,913.34","158,225,637.28"
49,2023-12-01,0.00,"14,270,261.48",0.00,0.00,"-6,484,200.84",0.00,"-2,658,930.56","5,127,130.08",0.00,...,"14,270,261.48","-9,143,131.40",0.00,0.00,0.10,98.65,0.99,"306,247,812.10","-142,895,044.74","163,352,767.36"
50,2024-01-01,0.00,"20,751,255.62","27,376.30",0.00,"-5,943,924.23",0.00,"-5,669,555.33","9,165,152.35",0.00,...,"20,778,631.92","-11,613,479.57",0.00,0.00,0.17,91.96,0.99,"327,026,444.02","-154,508,524.31","172,517,919.71"
51,2024-02-01,0.00,"29,691,662.12",0.00,0.00,"-4,567,709.04",0.00,"-1,581,115.17","23,542,837.91",0.00,...,"29,691,662.12","-6,148,824.21",0.00,0.00,0.35,74.02,0.99,"356,718,106.13","-160,657,348.51","196,060,757.62"
52,2024-03-01,0.00,"18,130,771.84","4,623.88",0.00,"-7,076,544.94",0.00,"-3,857,879.46","7,200,971.32",0.00,...,"18,135,395.72","-10,934,424.40",0.00,0.00,0.11,68.66,0.99,"374,853,501.85","-171,591,772.91","203,261,728.94"


In [36]:
# Percent Changes in account balances
window_size = 3  # Three months
# Calculate rolling averages and volatilities based on percent changes
for col in ['Total Revenues', 'Total Expenses', '1.9 - Net Income']:
    pct_chg_col = f'{col}_pct_chg'  # Define the percent change column name
    pivoted_income_statement[pct_chg_col] = pivoted_income_statement[col].pct_change()
    # Use the pct_chg_col variable correctly now
    pivoted_income_statement[f'{col}_rolling_avg_pct_chg'] = pivoted_income_statement[pct_chg_col].rolling(window=window_size).mean()
    pivoted_income_statement[f'{col}_volatility_pct_chg'] = pivoted_income_statement[pct_chg_col].rolling(window=window_size).std()
    for lag in range(1,13):
        pivoted_income_statement[f'{col}_rolling_avg_pct_chg_lag_{lag}'] = pivoted_income_statement[f'{col}_rolling_avg_pct_chg'].shift(lag)
        pivoted_income_statement[f'{col}_volatility_pct_chg_lag_{lag}'] = pivoted_income_statement[f'{col}_volatility_pct_chg'].shift(lag)


In [37]:

# Generate lagged features
for lag in range(1, 13):  # From 1 to 12 months
    pivoted_income_statement[f'Total_Revenues_Lag_{lag}'] = pivoted_income_statement['Total Revenues'].shift(lag)
    pivoted_income_statement[f'Total_Expenses_Lag_{lag}'] = pivoted_income_statement['Total Expenses'].shift(lag)
    pivoted_income_statement[f'Net_Income_Lag_{lag}'] = pivoted_income_statement['1.9 - Net Income'].shift(lag)
    pivoted_income_statement[f'profit_margin_Lag_{lag}'] = pivoted_income_statement['profit_margin'].shift(lag)
    pivoted_income_statement[f'ROA_Lag_{lag}'] = pivoted_income_statement['ROA'].shift(lag)
    pivoted_income_statement[f'ROE_Lag_{lag}'] = pivoted_income_statement['ROE'].shift(lag)
    pivoted_income_statement[f'debt_to_equity_Lag_{lag}'] = pivoted_income_statement['debt_to_equity'].shift(lag)
    pivoted_income_statement[f'debt_ratio_Lag_{lag}'] = pivoted_income_statement['debt_ratio'].shift(lag)


C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_30940\259549745.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  pivoted_income_statement[f'debt_ratio_Lag_{lag}'] = pivoted_income_statement['debt_ratio'].shift(lag)
C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_30940\259549745.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  pivoted_income_statement[f'Total_Revenues_Lag_{lag}'] = pivoted_income_statement['Total Revenues'].shift(lag)
C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_30940\259549745.py:4: PerformanceWa

In [282]:
pivoted_income_statement.shape[0]

53

Assets/Revenue Per Type
Coinbase asset type: http://forum.makerdao.com/t/mip81-coinbase-usdc-institutional-rewards/17703/254?u=sebventures


In [39]:
#assets_raw = dune.get_latest_result(58495)

In [40]:
#assets_p_t_ts = pd.DataFrame(assets_raw.result.rows)
#assets_p_t_ts['dt'] = pd.to_datetime(assets_p_t_ts['dt'])
#assets_p_t_ts.set_index('dt', inplace=True)

In [41]:
#assets_p_t_ts.head()

In [42]:
#assets_p_t_ts.to_csv(as_path)

In [48]:
as_path = '../data/csv/as.csv'

In [49]:
as_csv = pd.read_csv(as_path, index_col='dt', parse_dates=True)

In [50]:
as_csv = as_csv.drop(columns=['total_asset'])

Daily Interest Revenues By Vault

In [51]:
#ir_v = fetch_dune_data(3567939) 

In [52]:
#ir_v['period'] = pd.to_datetime(ir_v['period'])
#ir_v.set_index('period', inplace=True)

In [53]:
#ir_v.head()

In [54]:
daily_int_path = '../data/csv/d_int.csv'

In [55]:
#ir_v.to_csv(daily_int_path)

In [56]:
ir_csv = pd.read_csv(daily_int_path, index_col='period', parse_dates=True)

In [283]:
ir_csv.shape[0]

22031

In [58]:

ir_csv = ir_csv.rename_axis('day')

 

In [59]:
ir_csv = ir_csv.rename(columns={'collateral':'ilk'})

In [60]:
ir_csv['ilk'].unique()

array(['ETH-A', 'ETH-B', 'ETH-C', 'WBTC-A', 'WBTC-B', 'WBTC-C',
       'WSTETH-A', 'WSTETH-B', 'RWA002-A', 'RWA013-A', 'DIRECT-SPARK-DAI',
       'RWA014-A', 'RWA005-A', 'RWA012-A', 'RWA015-A', 'RWA007-A',
       'RETH-A', 'RWA003-A', 'GUNIV3DAIUSDC2-A', 'CRVV1ETHSTETH-A',
       'USDC-B', 'LINK-A', 'MATIC-A', 'UNIV2USDCETH-A', 'GNO-A',
       'UNIV2DAIUSDC-A', 'YFI-A', 'RWA004-A', 'GUNIV3DAIUSDC1-A',
       'GUSD-A', 'PAXUSD-A', 'USDC-A', 'DIRECT-AAVEV2-DAI',
       'DIRECT-COMPV2-DAI', 'RWA008-A', 'RENBTC-A', 'MANA-A', 'RWA009-A',
       'RWA001-A', 'UNI-A', 'UNIV2DAIETH-A', 'UNIV2WBTCETH-A',
       'UNIV2WBTCDAI-A', 'RWA-001', 'UNIV2UNIETH-A', 'TUSD-A', 'USDP-A',
       'BAT-A', 'BAL-A', 'ZRX-A', 'COMP-A', 'PSM-GUSD-A', 'AAVE-A',
       'UNIV2LINKETH-A', 'KNC-A', 'LRC-A', 'PSM-USDC-A', 'UNIV2AAVEETH-A',
       'UNIV2DAIUSDT-A', 'UNIV2ETHUSDT-A', 'USDT-A', 'PSM-PAX-A',
       'RWA006-A', 'PAX-A', nan, 'USDC', 'SAI'], dtype=object)

In [61]:
top_vaults = ir_csv.groupby('ilk').sum().sort_values('daily_revenues', ascending=False)

In [62]:
#10 Most Revenue Generating Vaults
top_10_vaults = top_vaults.head(10)

In [63]:
top_10_vaults

,daily_revenues
ilk,
ETH-A,"110,769,338.09"
WBTC-A,"41,088,135.60"
WSTETH-A,"22,414,721.27"
ETH-C,"17,297,706.33"
ETH-B,"15,374,386.83"
WSTETH-B,"11,554,717.49"
USDC-A,"7,351,192.96"
RWA013-A,"3,241,981.74"
WBTC-C,"2,697,915.32"


Dai Maturity Profile

Step-by-Step Process:
Tracking DAI Movements:

First, all transactions involving DAI are tracked to understand how DAI moves in and out of wallets. This includes both inflows (adding DAI to a wallet) and outflows (removing DAI from a wallet).
Defining Maturity Buckets:

Maturity buckets are predefined categories based on time durations, such as "1-day", "1-week", "1-month", "1-year", etc. Each bucket represents a hypothesis about how long DAI tends to stay put before being moved again.
Assigning Weights to Buckets:

Weights are assigned to each maturity bucket to reflect assumptions or historical observations about the distribution of DAI across these buckets. For example, if historically 30% of DAI is moved or used within a day, then the "1-day" bucket might get a weight of 0.30 (or 30%).
Applying Weights Based on Wallet Types:

DAI can be held in different types of wallets or contracts, each with its own expected behavior. For example, DAI in a savings contract (like DSR) might be considered more long-term ("1-year"), while DAI in a regular wallet might be more liquid ("1-day" or "1-week"). The weights applied to the DAI in these wallets reflect these expectations.
Calculating DAI Amounts per Bucket:

For each wallet (or DAI holding), the total amount of DAI is distributed across the maturity buckets based on the assigned weights. This means if a wallet has 100 DAI and the "1-day" bucket weight is 30%, then 30 DAI is considered to have a 1-day maturity.
The process is repeated for each wallet and each maturity bucket, based on the specific weights for that wallet type and the total DAI it holds.
Aggregating Across the Ecosystem:

Finally, to get the ecosystem-wide view, the amounts of DAI in each maturity bucket from all wallets are aggregated. This provides a snapshot of how much DAI is considered to be in each maturity bucket across the entire MakerDAO system at any given time.

In [64]:
#d_m = fetch_dune_data(907852)

In [65]:
#d_m['dt'] = pd.to_datetime(d_m['dt'])

In [66]:
#d_m

In [67]:

#d_m.to_csv(d_m_path)


In [68]:
d_m_path = '../data/csv/d_m.csv'

In [69]:
d_m_csv = pd.read_csv(d_m_path, index_col='dt', parse_dates=True)

In [284]:
d_m_csv.shape[0]

9528

In [71]:
dai_maturity_df = d_m_csv.pivot_table(
    index='dt',  # or 'period' if your DataFrame's time column is named 'period'
    columns='maturity',
    values=['outflow', 'outflow_dai_only', 'outflow_surplus_buffer'],
    aggfunc='sum'  # or any other aggregation function that fits your needs
)

# Flatten the MultiIndex columns (optional, for cleaner column names)
dai_maturity_df.columns = ['_'.join(col).strip() for col in dai_maturity_df.columns.values]

# Reset the index if you want 'dt' back as a regular column
dai_maturity_df.reset_index(inplace=True)



In [72]:
dai_maturity_df['dt'] = pd.to_datetime(dai_maturity_df['dt'])
dai_maturity_df.set_index('dt', inplace=True)


start_date = dai_maturity_df.index.min()
end_date = dai_maturity_df.index.max()
date_range = pd.date_range(start=start_date, end=end_date, freq='D')  # 'D' for daily frequency




In [73]:
dai_maturity_df_reindexed = dai_maturity_df.reindex(date_range)

# Forward-fill missing values
dai_maturity_df_reindexed.ffill(inplace=True)

# Optionally, reset the index if you want 'dt' back as a column
dai_maturity_df_reindexed.reset_index(inplace=True)
dai_maturity_df_reindexed.rename(columns={'index': 'day'}, inplace=True)

In [74]:
dai_maturity_df_reindexed.columns = [f'dai_maturity_{col}' if col != 'day' and not col.startswith('dai_maturity_') else col for col in dai_maturity_df_reindexed.columns]

In [75]:
dai_maturity_df_reindexed.tail()

,day,dai_maturity_outflow_1-block,dai_maturity_outflow_1-day,dai_maturity_outflow_1-month,dai_maturity_outflow_1-week,dai_maturity_outflow_1-year,dai_maturity_outflow_3-months,dai_maturity_outflow_dai_only_1-block,dai_maturity_outflow_dai_only_1-day,dai_maturity_outflow_dai_only_1-month,dai_maturity_outflow_dai_only_1-week,dai_maturity_outflow_dai_only_1-year,dai_maturity_outflow_dai_only_3-months,dai_maturity_outflow_surplus_buffer_1-block,dai_maturity_outflow_surplus_buffer_1-day,dai_maturity_outflow_surplus_buffer_1-month,dai_maturity_outflow_surplus_buffer_1-week,dai_maturity_outflow_surplus_buffer_1-year,dai_maturity_outflow_surplus_buffer_3-months
1585,2024-03-16 00:00:00+00:00,"453,757,132.17","453,757,132.17","136,277,964.30","591,913,771.82","2,850,066,310.40","201,065,848.96","453,757,132.17","453,757,132.17","136,277,964.30","591,913,771.82","2,781,519,501.85","201,065,848.96",0.00,0.00,0.00,0.00,"68,546,808.55",0.00
1586,2024-03-17 00:00:00+00:00,"455,307,007.70","455,307,007.70","137,425,908.66","596,032,698.73","2,849,349,624.78","202,759,537.36","455,307,007.70","455,307,007.70","137,425,908.66","596,032,698.73","2,780,461,988.11","202,759,537.36",0.00,0.00,0.00,0.00,"68,887,636.67",0.00
1587,2024-03-18 00:00:00+00:00,"461,095,932.81","461,095,932.81","140,439,203.63","607,501,207.95","2,863,252,281.53","207,205,382.40","461,095,932.81","461,095,932.81","140,439,203.63","607,501,207.95","2,796,227,598.03","207,205,382.40",0.00,0.00,0.00,0.00,"67,024,683.50",0.00
1588,2024-03-19 00:00:00+00:00,"455,524,838.32","455,524,838.32","137,117,144.96","595,167,015.97","2,854,314,452.33","202,303,984.36","455,524,838.32","455,524,838.32","137,117,144.96","595,167,015.97","2,787,586,179.26","202,303,984.36",0.00,0.00,0.00,0.00,"66,728,273.07",0.00
1589,2024-03-20 00:00:00+00:00,"451,479,837.89","451,479,837.89","135,641,898.57","589,090,233.01","2,833,931,375.49","200,127,391.33","451,479,837.89","451,479,837.89","135,641,898.57","589,090,233.01","2,766,819,019.01","200,127,391.33",0.00,0.00,0.00,0.00,"67,112,356.48",0.00


MakerDAO Stablecoin Ratio
This can give insights into the proportion of assets held in stablecoins (including DAI) relative to other assets. A higher stablecoin ratio might suggest a preference for stability within the MakerDAO system, which can have implications for DAI's stability.

In [76]:
#stablecoin_ratio_df = fetch_dune_data(58136)

In [77]:
#stablecoin_ratio_df['dt'] = pd.to_datetime(stablecoin_ratio_df['dt'])
#stablecoin_ratio_df.set_index('dt', inplace=True)

In [78]:
#stablecoin_ratio_df.to_csv(stablecoin_ratio_path)

In [79]:
stablecoin_ratio_path = '../data/csv/srp.csv'

In [80]:
stablecoin_ratio_csv = pd.read_csv(stablecoin_ratio_path, index_col='dt', parse_dates=True)

In [81]:
stablecoin_ratio_csv.head()

,Unnamed: 0,,assets_size,assets_stablecoins,assets_usdc,stablecoins_ratio,usdc_ratio
dt,,,,,,,
2024-03-21 00:00:00+00:00,0,1.00,"4,692,077,547.49","510,492,745.88","509,792,133.98",0.11,0.11
2024-03-20 00:00:00+00:00,1,1.00,"4,665,516,334.77","451,367,053.87","450,666,441.97",0.10,0.10
2024-03-19 00:00:00+00:00,2,1.00,"4,703,718,034.84","404,362,171.82","403,655,435.55",0.09,0.09
2024-03-18 00:00:00+00:00,3,1.00,"4,744,355,701.71","399,931,319.66","399,224,583.39",0.08,0.08
2024-03-17 00:00:00+00:00,4,1.00,"4,699,947,545.51","632,391,001.19","631,684,264.92",0.13,0.13


In [82]:
stable_coin_ratios = stablecoin_ratio_csv[['stablecoins_ratio','usdc_ratio']]

In [83]:
stable_coin_ratios = stable_coin_ratios.rename_axis('day')

In [84]:
stable_coin_ratios.index

DatetimeIndex(['2024-03-21 00:00:00+00:00', '2024-03-20 00:00:00+00:00',
               '2024-03-19 00:00:00+00:00', '2024-03-18 00:00:00+00:00',
               '2024-03-17 00:00:00+00:00', '2024-03-16 00:00:00+00:00',
               '2024-03-15 00:00:00+00:00', '2024-03-14 00:00:00+00:00',
               '2024-03-13 00:00:00+00:00', '2024-03-12 00:00:00+00:00',
               ...
               '2019-11-27 00:00:00+00:00', '2019-11-26 00:00:00+00:00',
               '2019-11-25 00:00:00+00:00', '2019-11-24 00:00:00+00:00',
               '2019-11-23 00:00:00+00:00', '2019-11-22 00:00:00+00:00',
               '2019-11-21 00:00:00+00:00', '2019-11-20 00:00:00+00:00',
               '2019-11-19 00:00:00+00:00', '2019-11-18 00:00:00+00:00'],
              dtype='datetime64[ns, UTC]', name='day', length=1586, freq=None)

Maker Peg Stability Module Stats
Data on the Peg Stability Module, which helps maintain DAI's peg to the USD, can be vital. Insights into the inflows, outflows, and balances within the PSM can directly indicate efforts to stabilize DAI.

In [85]:
#psm_stats_df = fetch_dune_data(17216)

In [86]:
#psm_stats_df['date'] = pd.to_datetime(psm_stats_df['date'])

In [87]:
psm_stats_path = '../data/csv/psm.csv'

In [88]:
#psm_stats_df.to_csv(psm_stats_path)

In [89]:
psm_stats_csv = pd.read_csv(psm_stats_path, index_col='date', parse_dates=True)

In [90]:
psm_stats_csv.columns = [f'psm_{col}' if col != 'dt' and not col.startswith('psm_') else col for col in psm_stats_csv.columns]


In [91]:
psm_stats_csv = psm_stats_csv.drop(columns=['psm_Unnamed: 0'])

In [92]:
psm_stats_csv = psm_stats_csv.rename_axis('day')

In [313]:
psm_stats_csv['psm_balance'].describe()

count           1,179.00
mean    2,495,919,311.99
std     1,628,328,770.12
min                 1.00
25%       673,188,300.21
50%     3,041,213,026.50
75%     3,652,875,373.77
max     5,965,535,187.98
Name: psm_balance, dtype: float64

Where is dai lets us know how dai is being used; dai in lending could be considered to add to stability 

In [94]:
#where_is_dai_df = fetch_dune_data(54599)

In [95]:
#where_is_dai_df['dt'] = pd.to_datetime(where_is_dai_df['dt'])

In [96]:
wid_path = '../data/csv/wid.csv'

In [97]:
#where_is_dai_df.to_csv(wid_path)

In [98]:
where_is_dai_csv = pd.read_csv(wid_path, index_col='dt', parse_dates=True)

In [99]:
where_is_dai_csv = where_is_dai_csv.rename_axis('day')

In [100]:
where_is_dai_csv.columns = [f'where_is_dai_{col}' if col != 'day' and not col.startswith('where_is_dai_') else col for col in where_is_dai_csv.columns]

In [101]:
where_is_dai_csv = where_is_dai_csv.drop(columns=['where_is_dai_Unnamed: 0'])

In [286]:
where_is_dai_csv.shape[0]

11112

In [103]:
where_is_dai_csv_table = where_is_dai_csv.pivot_table(values='where_is_dai_balance', index='day', columns='where_is_dai_wallet', aggfunc='sum')

In [306]:
where_is_dai_csv_table.describe()

,where_is_dai_Bridge,where_is_dai_CeFi,where_is_dai_Dai Savings,where_is_dai_Dex,where_is_dai_EOA,where_is_dai_Lending,where_is_dai_Other,where_is_dai_Treasury
count,"1,389.00","1,389.00","1,389.00","1,389.00","1,389.00","1,389.00","1,389.00","1,389.00"
mean,"519,571,584.34","97,793,318.99","311,612,990.83","759,685,160.26","2,313,033,791.12","583,815,304.12","384,965,889.61","18,564,924.60"
std,"479,488,038.35","78,415,911.28","518,564,427.53","660,249,624.37","1,133,804,050.66","565,104,617.62","199,111,750.50","22,963,663.97"
min,91.54,"6,032,365.70","1,167,096.23","5,969,545.93","70,539,903.52","2,382,557.63","7,624,088.20",0.00
25%,"254,354,292.48","22,517,650.67","12,215,114.08","242,906,003.63","1,797,171,880.58","86,534,442.33","256,075,295.20",0.00
50%,"452,459,560.65","83,466,578.34","90,963,507.53","563,571,678.17","2,869,541,236.00","379,804,255.54","384,559,660.50","19,322,535.41"
75%,"611,193,769.11","161,210,994.77","253,531,898.08","1,035,302,796.12","3,126,200,483.96","1,051,192,009.32","539,753,467.21","27,500,346.82"
max,"2,131,833,654.66","311,873,211.38","1,750,756,886.52","2,603,174,883.49","4,078,699,922.32","1,854,438,972.60","829,237,365.55","155,646,703.53"


In [105]:
where_is_dai_csv_table.columns = [f'where_is_dai_{col}' if col != 'day' and not col.startswith('where_is_dai_') else col for col in where_is_dai_csv_table.columns]

In [287]:
where_is_dai_csv_table.shape[0]

1389

Daily surplus buffer
Provides information on the surplus buffer in MakerDAO, which is a key financial metric. The surplus buffer acts as a reserve to cover potential system shortfalls and ensures the stability and solvency of the system. This data could be valuable for understanding the financial health and risk management strategies of MakerDAO over time

In [107]:
#daily_surplus_buffer = fetch_dune_data(3567837)

In [108]:
#daily_surplus_buffer['period'] = pd.to_datetime(daily_surplus_buffer['period'])

In [109]:
dsb_path = '../data/csv/dsb.csv'
#daily_surplus_buffer.to_csv(dsb_path)
daily_surplus_buffer_csv = pd.read_csv(dsb_path, index_col='period', parse_dates=True)

In [110]:
daily_surplus_buffer_csv = daily_surplus_buffer_csv.drop(columns=['Unnamed: 0'])

In [314]:
daily_surplus_buffer_csv.describe()

,daily_surplus_buffer_delta_30d,daily_surplus_buffer_delta_90d,daily_surplus_buffer
count,"1,609.00","1,609.00","1,609.00"
mean,"11,130,093.66","10,451,707.83","42,392,130.19"
std,"52,938,741.60","38,867,971.04","30,073,000.80"
min,"-210,344,528.52","-93,067,840.04","-5,674,219.91"
25%,"-10,263,499.22","-9,109,403.83","4,001,728.36"
50%,"2,130,021.51","2,024,616.70","50,260,545.40"
75%,"41,086,534.67","30,897,105.18","69,818,697.29"
max,"154,842,096.11","122,533,492.58","83,553,170.97"


In [112]:
daily_surplus_buffer_csv.columns = [f'daily_surplus_buffer_{col}' if col != 'period' and not col.startswith('surplus_buffer') else col for col in daily_surplus_buffer_csv.columns]

In [113]:
daily_surplus_buffer_csv = daily_surplus_buffer_csv.rename(columns={'surplus_buffer':'daily_surplus_buffer'})


In [114]:
daily_surplus_buffer_csv = daily_surplus_buffer_csv.rename_axis('day')

In [115]:
daily_surplus_buffer_csv.head()

,daily_surplus_buffer_delta_30d,daily_surplus_buffer_delta_90d,daily_surplus_buffer
day,,,
2024-03-27 00:00:00+00:00,"-118,286,288.53","-23,073,880.40","42,679,254.97"
2024-03-26 00:00:00+00:00,"-117,968,788.93","-22,860,798.30","42,972,541.24"
2024-03-25 00:00:00+00:00,"-114,559,431.40","-21,963,371.46","43,449,261.66"
2024-03-24 00:00:00+00:00,"-110,361,924.94","-20,375,056.07","44,097,497.35"
2024-03-23 00:00:00+00:00,"-105,856,419.40","-19,278,637.62","44,733,525.24"


Stability Fee history and Vault History - Rates Set by DAO
dart = debt balance

In [116]:
#sf_df = fetch_dune_data(3551110)

In [117]:
#sf_df['period'] = pd.to_datetime(sf_df['period'])

In [118]:
sf_path = '../data/csv/sf.csv'
#sf_df.to_csv(sf_path)
sf_history_csv = pd.read_csv(sf_path, index_col='period', parse_dates=True)

In [119]:
sf_history_csv

,Unnamed: 0,annualized,annualized_revenues,dart,ilk,total_ann_revenues
period,,,,,,
2024-03-21 00:00:00+00:00,0,0.05,0.00,0.00,GNO-A,"176,643,660.32"
2024-03-21 00:00:00+00:00,1,0.00,"42,545.93","70,910,068.44",GUNIV3DAIUSDC2-A,"176,643,660.32"
2024-03-21 00:00:00+00:00,2,0.03,-0.00,-0.00,LINK-A,"176,643,660.32"
2024-03-21 00:00:00+00:00,3,0.01,-0.00,-0.00,YFI-A,"176,643,660.32"
2024-03-21 00:00:00+00:00,4,0.01,0.00,0.00,COMP-A,"176,643,660.32"
...,...,...,...,...,...,...
2019-11-01 00:00:00+00:00,95411,NaN,NaN,NaN,WBTC-A,NaN
2019-11-01 00:00:00+00:00,95412,NaN,NaN,NaN,UNIV2UNIETH-A,NaN
2019-11-01 00:00:00+00:00,95413,NaN,NaN,NaN,RWA005-A,NaN


In [120]:
sf_history_csv_reset = sf_history_csv.reset_index()

In [121]:
sf_history_csv_clean = sf_history_csv_reset.drop_duplicates(subset=['period', 'ilk'], keep='last')

In [122]:
sf_history_csv_clean = sf_history_csv_clean.drop(columns='Unnamed: 0')

In [123]:
sf_history_csv_clean['period'] = sf_history_csv_clean['period'].dt.date

In [124]:
sf_history_csv_clean = sf_history_csv_clean.rename(columns={'period':'day'})

DAI Savings Rate Historical - Set by DAO

In [125]:
#dsr_rate = fetch_dune_data(3581248)

In [126]:
dsr_rate_path = '../data/csv/dsr.csv'
#dsr_rate.to_csv(dsr_rate_path)
dsr_rate_csv = pd.read_csv(dsr_rate_path, index_col='dt', parse_dates=True)

In [127]:
dsr_rate_csv['dsr_rate'].describe()

count   1,552.00
mean        0.01
std         0.03
min         0.00
25%         0.00
50%         0.00
75%         0.01
max         0.15
Name: dsr_rate, dtype: float64

In [128]:
dsr_rate_csv = dsr_rate_csv.drop(columns=['Unnamed: 0'])

In [129]:
dsr_rate_csv['dai_percent_in_dsr'] = dsr_rate_csv['dsr_balance'] / dsr_rate_csv['total_balance']
dsr_rate_csv['dai_percent_out_dsr'] = dsr_rate_csv['non_dsr_balance'] / dsr_rate_csv['total_balance']

In [130]:
dsr_rate_csv = dsr_rate_csv.rename_axis('day')

In [289]:
dsr_rate_csv.shape[0]

1552

dsr flows

In [132]:
#dsr_flows = fetch_dune_data(1753750)

In [133]:
dsr_flows_path='../data/csv/dsr_flows.csv'
#dsr_flows.to_csv(dsr_flows_path)
dsr_flows_csv = pd.read_csv(dsr_flows_path, index_col='period', parse_dates=True)

In [290]:
dsr_flows_csv.shape[0]

581

In [135]:
dsr_flows_csv = dsr_flows_csv.drop(columns=['Unnamed: 0','balance'])
dsr_flows_csv = dsr_flows_csv.rename_axis('day')

In [136]:
dsr_flows_csv.columns = [f'dsr_{col}' if col != 'day' and not col.startswith('surplus_buffer') else col for col in dsr_flows_csv.columns]

In [137]:
dsr_df = dsr_flows_csv.merge(dsr_rate_csv, on=['day'], how='inner')

In [138]:
dsr_df = dsr_df.rename(columns={'total_balance':'dai_total_balance'})
dsr_df = dsr_df.rename(columns={'non_dsr_balance':'dai_circulating'})

In [315]:
dsr_df.describe()

,day,dsr_inflow,dsr_interest,dsr_outflow,dsr_balance,dsr_rate,dai_circulating,dai_total_balance,dai_percent_in_dsr,dai_percent_out_dsr
count,581,581.00,581.00,581.00,581.00,581.00,581.00,581.00,581.00,581.00
mean,2023-06-11 00:00:00,"6,237,512.18","92,652.68","3,794,905.09","632,212,219.96",0.03,"4,584,444,424.95","5,216,656,644.91",0.12,0.88
min,2022-08-25 00:00:00,0.00,0.00,0.00,"1,167,096.23",0.00,"3,182,053,005.07","4,336,565,083.61",0.00,0.67
25%,2023-01-17 00:00:00,0.00,0.00,0.00,"35,775,627.77",0.00,"3,756,055,150.83","4,854,191,351.00",0.01,0.72
50%,2023-06-11 00:00:00,0.00,"9,502.93",0.00,"109,586,897.55",0.01,"4,542,041,297.79","5,174,925,449.48",0.02,0.98
75%,2023-11-03 00:00:00,"2,202,044.99","197,852.53",0.00,"1,434,744,378.05",0.05,"5,136,076,330.01","5,391,627,556.52",0.28,0.99
max,2024-03-27 00:00:00,"298,378,513.73","591,988.67","337,868,152.03","1,750,756,886.52",0.15,"6,549,164,445.71","6,550,365,936.14",0.33,1.00
std,NaN,"21,730,030.76","122,013.21","21,526,774.49","689,782,183.06",0.03,"895,024,557.86","495,795,966.59",0.13,0.13


In [140]:
#cum_bal_and_safetyprice_and_safetyvalue = vault_stats_6_20_through_6_21 

In [141]:
cumbal_stats_path ='../data/csv/cumbal.csv'
#cum_bal_and_safetyprice_and_safetyvalue.to_csv(cumbal_stats_path)
cumbal_csv = pd.read_csv(cumbal_stats_path, index_col = 'day', parse_dates=True)

In [142]:
#debtbal_lpenalty_lratio = fetch_dune_data(3568425)

In [143]:
debtbal_lpenalty_lratio_path = '../data/csv/debtbal_lpenalty_lratio.csv'
#debtbal_lpenalty_lratio.to_csv(debtbal_lpenalty_lratio_path)
debtbal_lpenalty_lratio_path_csv = pd.read_csv(debtbal_lpenalty_lratio_path, index_col = 'day', parse_dates=True)

In [144]:
debtbal_lpenalty_lratio_path_csv.head()

,Unnamed: 0,debt_balance,ilk,liquidation_penalty,liquidation_ratio
day,,,,,
2024-03-27,0,0.00,BAL-A,0.00,7.48
2024-03-27,1,-0.00,KNC-A,0.00,12.28
2024-03-27,2,"300,484,322.01",ETH-C,0.13,1.70
2024-03-27,3,0.00,GUSD-A,0.13,15.00
2024-03-27,4,0.00,BAT-A,0.00,47.50


In [145]:
debtbal_lpenalty_lratio_path_csv_reset = debtbal_lpenalty_lratio_path_csv.reset_index()
debtbal_lpenalty_lratio_path_csv_clean = debtbal_lpenalty_lratio_path_csv_reset.drop_duplicates(subset=['day', 'ilk'], keep='last')

In [146]:
debtbal_lpenalty_lratio_path_csv_clean = debtbal_lpenalty_lratio_path_csv_clean.drop(columns='Unnamed: 0')

In [147]:
#dceiling_dfloor_scratio = fetch_dune_data(3568438)

In [148]:
dceiling_dfloor_scratio_path = '../data/csv/dceiling_dfloor_scratio.csv'
#dceiling_dfloor_scratio.to_csv(dceiling_dfloor_scratio_path)
dceiling_dfloor_scratio_csv = pd.read_csv(dceiling_dfloor_scratio_path, index_col = 'day', parse_dates=True)

In [149]:
#vault_market_price = fetch_dune_data(3568453)

In [150]:
vault_market_price_path = '../data/csv/vault_market_price.csv'
#vault_market_price.to_csv(vault_market_price_path)
vault_market_price_csv = pd.read_csv(vault_market_price_path, index_col = 'day', parse_dates=True)

In [151]:
vault_market_price_csv.head()

,Unnamed: 0,ilk,market_price
day,,,
2024-03-27,0,GUNIV3DAIUSDC2-A,203.39
2024-03-27,1,BAL-A,21.64
2024-03-27,2,LINK-A,6.28
2024-03-27,3,UNIV2WBTCDAI-A,"31,062,885.00"
2024-03-27,4,WBTC-A,"69,767.86"


In [152]:
vault_market_price_csv_reset = vault_market_price_csv.reset_index()
vault_market_price_csv_clean = vault_market_price_csv_reset.drop_duplicates(subset=['day', 'ilk'], keep='last')

In [153]:
vault_market_price_csv_clean = vault_market_price_csv_clean.drop(columns=['Unnamed: 0'])

In [154]:
cumbal_csv_reset = cumbal_csv.reset_index()
dceiling_dfloor_scratio_csv_reset = dceiling_dfloor_scratio_csv.reset_index()

In [155]:
cumbal_csv_clean = cumbal_csv_reset.drop_duplicates(subset=['day', 'ilk'], keep='last')
dceiling_dfloor_scratio_csv_clean = dceiling_dfloor_scratio_csv_reset.drop_duplicates(subset=['day', 'ilk'], keep='last')

In [156]:
cumbal_csv_clean = cumbal_csv_clean.drop(columns=['Unnamed: 0'])

In [157]:
dceiling_dfloor_scratio_csv_clean = dceiling_dfloor_scratio_csv_clean.drop(columns=['Unnamed: 0'])

In [158]:
comprehensive_vault_stats = pd.merge(cumbal_csv_clean, dceiling_dfloor_scratio_csv_clean, on=['day', 'ilk'], how='inner')

In [159]:
comprehensive_vault_stats = comprehensive_vault_stats.merge(vault_market_price_csv_clean, on=['day', 'ilk'], how='inner' )

In [160]:
comprehensive_vault_stats = comprehensive_vault_stats.merge(debtbal_lpenalty_lratio_path_csv_clean, on=['day', 'ilk'], how='inner')

In [161]:
comprehensive_vault_stats['day'] = pd.to_datetime(comprehensive_vault_stats['day'])
sf_history_csv_clean['day'] = pd.to_datetime(sf_history_csv_clean['day'])

In [162]:
ir_csv.columns

Index(['ilk', 'daily_revenues'], dtype='object')

In [163]:
comprehensive_vault_stats.columns

Index(['day', 'cumulative_collateral', 'ilk', 'safety_price',
       'usd_safety_value', 'dai_ceiling', 'dai_floor',
       'safety_collateral_ratio', 'market_price', 'debt_balance',
       'liquidation_penalty', 'liquidation_ratio'],
      dtype='object')

In [164]:
comprehensive_vault_stats = comprehensive_vault_stats.merge(ir_csv, on=['day', 'ilk'], how='inner')

In [165]:
comprehensive_vault_stats = comprehensive_vault_stats.merge(sf_history_csv_clean, on=['day', 'ilk'], how='inner')

In [168]:
# Ensure both 'day' columns are timezone-aware and in UTC
comprehensive_vault_stats['day'] = comprehensive_vault_stats['day'].dt.tz_convert ('UTC')
dai_maturity_df_reindexed['day'] = dai_maturity_df_reindexed['day'].dt.tz_convert ('UTC')


In [169]:
stable_coin_ratios.index

DatetimeIndex(['2024-03-21 00:00:00+00:00', '2024-03-20 00:00:00+00:00',
               '2024-03-19 00:00:00+00:00', '2024-03-18 00:00:00+00:00',
               '2024-03-17 00:00:00+00:00', '2024-03-16 00:00:00+00:00',
               '2024-03-15 00:00:00+00:00', '2024-03-14 00:00:00+00:00',
               '2024-03-13 00:00:00+00:00', '2024-03-12 00:00:00+00:00',
               ...
               '2019-11-27 00:00:00+00:00', '2019-11-26 00:00:00+00:00',
               '2019-11-25 00:00:00+00:00', '2019-11-24 00:00:00+00:00',
               '2019-11-23 00:00:00+00:00', '2019-11-22 00:00:00+00:00',
               '2019-11-21 00:00:00+00:00', '2019-11-20 00:00:00+00:00',
               '2019-11-19 00:00:00+00:00', '2019-11-18 00:00:00+00:00'],
              dtype='datetime64[ns, UTC]', name='day', length=1586, freq=None)

In [170]:
comprehensive_vault_stats.tail()

,day,cumulative_collateral,ilk,safety_price,usd_safety_value,dai_ceiling,dai_floor,safety_collateral_ratio,market_price,debt_balance,liquidation_penalty,liquidation_ratio,daily_revenues,annualized,annualized_revenues,dart,total_ann_revenues
21747,2019-11-15 00:00:00+00:00,0.42,ETH-A,119.74,50.29,0.00,20.00,0.00,NaN,NaN,NaN,NaN,0.00,0.04,NaN,NaN,NaN
21748,2019-11-15 00:00:00+00:00,10.00,BAT-A,0.17,1.66,0.00,20.00,0.00,NaN,NaN,NaN,NaN,0.00,0.04,NaN,NaN,NaN
21749,2019-11-13 00:00:00+00:00,NaN,SAI,0.00,NaN,0.00,0.00,0.00,NaN,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN
21750,2019-11-13 00:00:00+00:00,0.42,ETH-A,0.00,0.00,0.00,20.00,0.00,NaN,NaN,NaN,NaN,0.00,0.04,NaN,NaN,NaN
21751,2019-11-13 00:00:00+00:00,10.00,BAT-A,0.00,0.00,0.00,20.00,0.00,NaN,NaN,NaN,NaN,0.00,0.04,NaN,NaN,NaN


In [171]:
comprehensive_vault_stats.columns

Index(['day', 'cumulative_collateral', 'ilk', 'safety_price',
       'usd_safety_value', 'dai_ceiling', 'dai_floor',
       'safety_collateral_ratio', 'market_price', 'debt_balance',
       'liquidation_penalty', 'liquidation_ratio', 'daily_revenues',
       'annualized', 'annualized_revenues', 'dart', 'total_ann_revenues'],
      dtype='object')

In [172]:
comprehensive_vault_stats = comprehensive_vault_stats.rename(columns={'annualized':'annualized stability fee'})

In [173]:
def determine_status(row):
    # The vault is considered closed if 'dai_ceiling' is 0
    # This includes the first day if 'dai_ceiling' starts at 0 or if it drops to 0 from a nonzero value
    if pd.isnull(row['prev_dai_ceiling']) and row['dai_ceiling'] == 0:
        return 'Closed'
    elif row['prev_dai_ceiling'] >= 0 and row['dai_ceiling'] == 0:
        return 'Closed'
    # Check if 'safety_price' is 0
    elif row['safety_price'] == 0:
        return 'Closed'
    else:
        return 'Open'

In [174]:
comprehensive_vault_stats['market_price'] = np.where(comprehensive_vault_stats['liquidation_ratio'].isnull(), comprehensive_vault_stats['safety_price'], comprehensive_vault_stats['market_price'])

In [175]:
comprehensive_vault_stats = comprehensive_vault_stats.sort_values('day')
comprehensive_vault_stats['prev_dai_ceiling'] = comprehensive_vault_stats.groupby('ilk')['dai_ceiling'].shift(1)

In [176]:
comprehensive_vault_stats['status'] = comprehensive_vault_stats.apply(determine_status, axis=1)


In [177]:
comprehensive_vault_stats.columns

Index(['day', 'cumulative_collateral', 'ilk', 'safety_price',
       'usd_safety_value', 'dai_ceiling', 'dai_floor',
       'safety_collateral_ratio', 'market_price', 'debt_balance',
       'liquidation_penalty', 'liquidation_ratio', 'daily_revenues',
       'annualized stability fee', 'annualized_revenues', 'dart',
       'total_ann_revenues', 'prev_dai_ceiling', 'status'],
      dtype='object')

In [178]:
comprehensive_vault_stats['market_collateral_ratio'] = np.where(comprehensive_vault_stats['status']=='Open',((comprehensive_vault_stats['usd_safety_value'] *comprehensive_vault_stats['liquidation_ratio']) / comprehensive_vault_stats['debt_balance']), np.nan)
comprehensive_vault_stats['market_collateral_ratio'] = np.where(comprehensive_vault_stats['debt_balance']==0,0, comprehensive_vault_stats['market_collateral_ratio'])

comprehensive_vault_stats['safety_collateral_ratio'] = np.where(comprehensive_vault_stats['status']=='Open',comprehensive_vault_stats['safety_collateral_ratio'], np.nan)
comprehensive_vault_stats['safety_collateral_ratio'] = np.where(comprehensive_vault_stats['status'] == 'Closed', 0, comprehensive_vault_stats['safety_collateral_ratio'])
comprehensive_vault_stats['safety_collateral_ratio'] = np.where(comprehensive_vault_stats['debt_balance'] <= 0, 0, comprehensive_vault_stats['safety_collateral_ratio'])
comprehensive_vault_stats[['annualized_revenues', 'dart']] = np.where(comprehensive_vault_stats[['annualized_revenues', 'dart']] <= 0, 0, comprehensive_vault_stats[['annualized_revenues', 'dart']])

comprehensive_vault_stats['collateral_usd'] = comprehensive_vault_stats['usd_safety_value'] * comprehensive_vault_stats['liquidation_ratio']
comprehensive_vault_stats['hypothetical_dai_ceiling'] = comprehensive_vault_stats['collateral_usd']* (comprehensive_vault_stats['liquidation_ratio'] / 2)

In [179]:
#replace values when vault is closed to 0?

In [180]:
def clean_small_values(value, threshold=1e-8):
    """
    Set small negative and positive values within a specified threshold to zero.

    Parameters:
    - value: The value to be cleaned.
    - threshold: Values within +/- this threshold will be set to zero.

    Returns:
    - The cleaned value.
    """
    if -threshold < value < threshold:
        return 0.0
    else:
        return value

# Apply this function to the entire DataFrame
comprehensive_vault_stats[['cumulative_collateral','usd_safety_value','collateral_usd','hypothetical_dai_ceiling','debt_balance']] = comprehensive_vault_stats[['cumulative_collateral','usd_safety_value','collateral_usd','hypothetical_dai_ceiling','debt_balance']].applymap(lambda x: clean_small_values(x))


C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_30940\4189237013.py:18: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  comprehensive_vault_stats[['cumulative_collateral','usd_safety_value','collateral_usd','hypothetical_dai_ceiling','debt_balance']] = comprehensive_vault_stats[['cumulative_collateral','usd_safety_value','collateral_usd','hypothetical_dai_ceiling','debt_balance']].applymap(lambda x: clean_small_values(x))


In [181]:
comprehensive_vault_stats = comprehensive_vault_stats[comprehensive_vault_stats['ilk'] != 'SAI']

UNIV2ETHUSDT-A is an outlier and will be removed from dataset.  

In [182]:
comprehensive_vault_stats[comprehensive_vault_stats['safety_collateral_ratio'] < 0 ]

,day,cumulative_collateral,ilk,safety_price,usd_safety_value,dai_ceiling,dai_floor,safety_collateral_ratio,market_price,debt_balance,...,daily_revenues,annualized stability fee,annualized_revenues,dart,total_ann_revenues,prev_dai_ceiling,status,market_collateral_ratio,collateral_usd,hypothetical_dai_ceiling
12071,2021-08-08 00:00:00+00:00,0.00,UNIV2ETHUSDT-A,"137,635,170.15",0.00,"2,000,000.00","10,000.00",-12.95,"192,689,238.21",0.00,...,4.87,0.02,0.00,0.00,"38,032,976.74","2,006,530.97",Open,-18.13,0.00,0.00
12038,2021-08-09 00:00:00+00:00,0.00,UNIV2ETHUSDT-A,"139,168,140.36",0.00,"2,000,000.00","10,000.00",-13.09,"194,835,396.50",0.00,...,4.75,0.02,0.00,0.00,"38,121,293.97","2,000,000.00",Open,-18.33,0.00,0.00
11980,2021-08-10 00:00:00+00:00,0.00,UNIV2ETHUSDT-A,"140,063,017.89",0.00,"2,000,000.00","10,000.00",-13.18,"196,088,225.05",0.00,...,4.75,0.02,0.00,0.00,"38,691,927.83","2,000,000.00",Open,-18.45,0.00,0.00
11963,2021-08-11 00:00:00+00:00,0.00,UNIV2ETHUSDT-A,"141,384,297.67",0.00,"2,000,000.00","10,000.00",-13.30,"197,938,016.73",0.00,...,4.73,0.02,0.00,0.00,"38,773,987.68","2,000,000.00",Open,-18.62,0.00,0.00
11918,2021-08-12 00:00:00+00:00,0.00,UNIV2ETHUSDT-A,"137,867,966.00",0.00,"2,000,000.00","10,000.00",-12.97,"193,015,152.40",0.00,...,4.74,0.02,0.00,0.00,"40,333,600.87","2,000,000.00",Open,-18.16,0.00,0.00
11864,2021-08-13 00:00:00+00:00,0.00,UNIV2ETHUSDT-A,"141,650,852.35",0.00,"2,000,000.00","10,000.00",-13.33,"198,311,193.29",0.00,...,4.74,0.02,0.00,0.00,"39,855,556.08","2,000,000.00",Open,-18.66,0.00,0.00
11843,2021-08-14 00:00:00+00:00,0.00,UNIV2ETHUSDT-A,"142,284,082.28",0.00,"2,000,000.00","10,000.00",-13.39,"199,197,715.19",0.00,...,4.74,0.02,0.00,0.00,"40,109,465.52","2,000,000.00",Open,-18.74,0.00,0.00
11807,2021-08-15 00:00:00+00:00,0.00,UNIV2ETHUSDT-A,"139,997,139.81",0.00,"2,000,000.00","10,000.00",-13.17,"195,995,995.74",0.00,...,4.75,0.02,0.00,0.00,"40,161,597.87","2,000,000.00",Open,-18.44,0.00,0.00
11765,2021-08-16 00:00:00+00:00,0.00,UNIV2ETHUSDT-A,"142,001,139.88",0.00,"2,000,000.00","10,000.00",-13.36,"198,801,595.84",0.00,...,4.74,0.02,0.00,0.00,"40,275,499.78","2,000,000.00",Open,-18.70,0.00,0.00
11701,2021-08-17 00:00:00+00:00,0.00,UNIV2ETHUSDT-A,"140,896,250.11",0.00,"2,000,000.00","10,000.00",-13.26,"197,254,750.15",0.00,...,4.78,0.02,0.00,0.00,"40,012,479.05","2,000,000.00",Open,-18.56,0.00,0.00


In [183]:
comprehensive_vault_stats = comprehensive_vault_stats[comprehensive_vault_stats['ilk'] != 'UNIV2ETHUSDT-A']

In [184]:
comprehensive_vault_stats['debt_balance'].describe()

count          21,138.00
mean      138,522,602.01
std       387,265,830.49
min            -2,528.51
25%         1,403,187.58
50%        15,818,062.08
75%        88,412,143.20
max     3,361,006,014.83
Name: debt_balance, dtype: float64

In [185]:
#pd.set_option('display.max_columns', None)  # Show all columns
#pd.set_option('display.max_rows', None)  # Show all rows
#pd.set_option('display.max_colwidth', None)  # Show full content of each column

In [186]:
comprehensive_vault_stats['market_collateral_ratio'].describe()

count                    19,378.00
mean        160,876,285,659,620.75
std       7,377,037,911,158,396.00
min     -32,429,545,705,439,232.00
25%                           2.11
50%                           3.14
75%                           4.01
max     324,295,457,054,392,320.00
Name: market_collateral_ratio, dtype: float64

In [187]:
comprehensive_vault_stats['safety_collateral_ratio'].describe()

count                    21,501.00
mean        144,095,140,219,423.31
std       6,659,657,583,570,546.00
min                           0.00
25%                           1.41
50%                           1.94
75%                           2.50
max     308,852,816,242,278,400.00
Name: safety_collateral_ratio, dtype: float64

In [188]:
comprehensive_vault_stats[['day','ilk','market_collateral_ratio']][comprehensive_vault_stats['market_collateral_ratio'] > 3.31 ].sort_values('market_collateral_ratio')

,day,ilk,market_collateral_ratio
6980,2022-02-10 00:00:00+00:00,RENBTC-A,3.31
13243,2021-07-06 00:00:00+00:00,RENBTC-A,3.31
20676,2020-08-24 00:00:00+00:00,KNC-A,3.31
845,2023-12-07 00:00:00+00:00,ETH-B,3.31
2408,2023-05-19 00:00:00+00:00,RETH-A,3.31
...,...,...,...
7720,2022-01-05 00:00:00+00:00,RWA003-A,"324,295,457,054,392,320.00"
8489,2021-11-30 00:00:00+00:00,RWA003-A,"324,295,457,054,392,320.00"
8141,2021-12-18 00:00:00+00:00,RWA003-A,"324,295,457,054,392,320.00"
7463,2022-01-15 00:00:00+00:00,RWA003-A,"324,295,457,054,392,320.00"


In [189]:
comprehensive_vault_stats[['day','ilk','safety_collateral_ratio']][comprehensive_vault_stats['safety_collateral_ratio'] > 100 ]

,day,ilk,safety_collateral_ratio
15803,2021-04-21 00:00:00+00:00,RWA002-A,149.50
15727,2021-04-23 00:00:00+00:00,RWA001-A,"32,616,554.14"
15494,2021-04-30 00:00:00+00:00,RWA001-A,"32,616,554.14"
15464,2021-05-01 00:00:00+00:00,RWA001-A,"32,616,554.14"
15421,2021-05-02 00:00:00+00:00,RWA001-A,"32,616,554.14"
...,...,...,...
6015,2022-05-12 00:00:00+00:00,YFI-A,114.43
5992,2022-05-13 00:00:00+00:00,YFI-A,111.95
5637,2022-06-13 00:00:00+00:00,RWA003-A,113.28
4516,2022-09-30 00:00:00+00:00,RWA003-A,"9,651,650,507,571,200.00"


In [190]:
comprehensive_vault_stats[['day','ilk','debt_balance']][comprehensive_vault_stats['debt_balance'] < 0 ]

,day,ilk,debt_balance
3051,2023-03-11 00:00:00+00:00,DIRECT-AAVEV2-DAI,"-2,528.51"


In [191]:
comprehensive_vault_stats = comprehensive_vault_stats[comprehensive_vault_stats['ilk'] != 'DIRECT-AAVEV2-DAI']

In [192]:
comprehensive_vault_stats[['day','ilk','debt_balance']][comprehensive_vault_stats['debt_balance'] < 0 ]

,day,ilk,debt_balance


In [193]:
comprehensive_vault_stats['status'].describe()

count     21499
unique        2
top        Open
freq      19896
Name: status, dtype: object

In [194]:
comprehensive_vault_stats[['liquidation_ratio','liquidation_penalty','annualized stability fee',
       'annualized_revenues', 'dart', 'total_ann_revenues',
       'prev_dai_ceiling']].describe()

,liquidation_ratio,liquidation_penalty,annualized stability fee,annualized_revenues,dart,total_ann_revenues,prev_dai_ceiling
count,"21,004.00","15,665.00","21,493.00","21,136.00","21,136.00","21,491.00","21,445.00"
mean,1.49,-0.16,0.04,"3,167,088.52","138,541,671.93","72,830,970.89","173,188,464.60"
std,2.20,0.49,0.07,"10,267,730.84","387,325,542.99","62,927,201.18","470,887,247.03"
min,0.00,-1.00,0.00,0.00,0.00,767.83,0.00
25%,1.25,-1.00,0.01,"4,979.85","1,402,720.70","29,094,709.62","3,306,266.02"
50%,1.50,0.13,0.03,"172,814.73","15,890,305.95","52,913,177.34","20,000,000.00"
75%,1.75,0.13,0.05,"1,678,781.65","88,457,397.17","99,900,414.71","120,000,000.00"
max,100.00,0.13,0.50,"142,209,506.83","3,361,006,014.83","351,700,458.61","4,361,006,014.83"


In [195]:
closed_vaults = comprehensive_vault_stats[comprehensive_vault_stats['status'] == 'Closed']

In [196]:
zero_balances = comprehensive_vault_stats[comprehensive_vault_stats['cumulative_collateral'] == 0]

In [197]:
usdt_a = comprehensive_vault_stats[comprehensive_vault_stats['ilk']=='USDT-A']

In [198]:
usdt_a.head()

,day,cumulative_collateral,ilk,safety_price,usd_safety_value,dai_ceiling,dai_floor,safety_collateral_ratio,market_price,debt_balance,...,daily_revenues,annualized stability fee,annualized_revenues,dart,total_ann_revenues,prev_dai_ceiling,status,market_collateral_ratio,collateral_usd,hypothetical_dai_ceiling
20599,2020-09-08 00:00:00+00:00,368.90,USDT-A,0.67,246.01,"10,000,000.00",100.00,1.00,1.00,245.99,...,0.02,0.06,14.76,245.99,"1,855,497.42",NaN,Open,1.50,369.02,276.76
20588,2020-09-09 00:00:00+00:00,0.00,USDT-A,0.67,0.00,"10,000,000.00",100.00,0.00,1.00,0.00,...,0.00,0.06,0.00,0.00,"1,763,209.76","10,000,000.00",Open,0.00,0.00,0.00
20567,2020-09-12 00:00:00+00:00,200.00,USDT-A,0.67,133.38,"10,000,000.00",100.00,0.00,1.00,0.00,...,0.12,0.06,0.00,0.00,"2,227,239.39","10,000,000.00",Open,0.00,200.07,150.05
20561,2020-09-13 00:00:00+00:00,"426,826.22",USDT-A,0.67,"284,680.54","10,000,000.00",100.00,1.50,1.00,"189,858.91",...,20.63,0.06,"11,391.53","189,858.91","2,223,945.55","10,000,000.00",Open,2.25,"427,020.81","320,265.61"
20558,2020-09-14 00:00:00+00:00,"531,480.11",USDT-A,0.67,"354,757.69","10,000,000.00",100.00,1.42,1.00,"249,920.00",...,44.86,0.06,"14,995.20","249,920.00","3,317,027.83","10,000,000.00",Open,2.13,"532,136.54","399,102.40"


In [199]:
zero_balances['ilk'].unique()

array(['USDT-A', 'UNIV2WBTCDAI-A', 'UNIV2DAIUSDT-A', 'RWA008-A'],
      dtype=object)

In [200]:
closed_vaults['ilk'].unique()

array(['BAT-A', 'ETH-A', 'LRC-A', 'COMP-A', 'LINK-A', 'ETH-B', 'BAL-A',
       'GUSD-A', 'TUSD-A', 'UNIV2DAIUSDC-A', 'UNIV2AAVEETH-A',
       'UNIV2WBTCDAI-A', 'UNIV2DAIUSDT-A', 'USDC-A', 'USDT-A', 'PAXUSD-A',
       'USDC-B', 'KNC-A', 'RWA006-A', 'GUNIV3DAIUSDC1-A',
       'UNIV2LINKETH-A', 'ZRX-A', 'AAVE-A', 'CRVV1ETHSTETH-A',
       'UNIV2DAIETH-A', 'UNIV2WBTCETH-A', 'UNI-A', 'RWA009-A', 'RETH-A',
       'RENBTC-A', 'GNO-A', 'GUNIV3DAIUSDC2-A', 'UNIV2USDCETH-A',
       'MATIC-A', 'RWA003-A', 'RWA005-A'], dtype=object)

In [201]:
wbtc = comprehensive_vault_stats[comprehensive_vault_stats['ilk']=='WBTC-A']
eth_a = comprehensive_vault_stats[comprehensive_vault_stats['ilk']=='ETH-A']

In [202]:
eth_a['status']

21750    Closed
21747    Closed
21745    Closed
21743    Closed
21742      Open
          ...  
37         Open
28         Open
18         Open
16         Open
7          Open
Name: status, Length: 1564, dtype: object

In [203]:
eth_a_df = eth_a['dai_ceiling'].to_frame('eth_a_dai_ceiling')

In [204]:
eth_a_zero_dai_ceiling = eth_a[eth_a['dai_ceiling'] == 0]

In [205]:
eth_a_zero_dai_ceiling.head()

,day,cumulative_collateral,ilk,safety_price,usd_safety_value,dai_ceiling,dai_floor,safety_collateral_ratio,market_price,debt_balance,...,daily_revenues,annualized stability fee,annualized_revenues,dart,total_ann_revenues,prev_dai_ceiling,status,market_collateral_ratio,collateral_usd,hypothetical_dai_ceiling
21750,2019-11-13 00:00:00+00:00,0.42,ETH-A,0.00,0.00,0.00,20.00,0.00,0.00,NaN,...,0.00,0.04,NaN,NaN,NaN,NaN,Closed,NaN,NaN,NaN
21747,2019-11-15 00:00:00+00:00,0.42,ETH-A,119.74,50.29,0.00,20.00,0.00,119.74,NaN,...,0.00,0.04,NaN,NaN,NaN,0.00,Closed,NaN,NaN,NaN
21745,2019-11-16 00:00:00+00:00,0.42,ETH-A,121.69,51.11,0.00,20.00,0.00,121.69,NaN,...,0.00,0.04,NaN,NaN,NaN,0.00,Closed,NaN,NaN,NaN
21743,2019-11-17 00:00:00+00:00,0.43,ETH-A,123.31,53.02,0.00,20.00,0.00,123.31,NaN,...,0.00,0.04,NaN,NaN,NaN,0.00,Closed,NaN,NaN,NaN


In [206]:
eth_a_df.head()

,eth_a_dai_ceiling
21750,0.00
21747,0.00
21745,0.00
21743,0.00
21742,"50,000,000.00"


In [207]:
wbtc = wbtc.set_index('day')

In [208]:
comprehensive_vault_stats.columns

Index(['day', 'cumulative_collateral', 'ilk', 'safety_price',
       'usd_safety_value', 'dai_ceiling', 'dai_floor',
       'safety_collateral_ratio', 'market_price', 'debt_balance',
       'liquidation_penalty', 'liquidation_ratio', 'daily_revenues',
       'annualized stability fee', 'annualized_revenues', 'dart',
       'total_ann_revenues', 'prev_dai_ceiling', 'status',
       'market_collateral_ratio', 'collateral_usd',
       'hypothetical_dai_ceiling'],
      dtype='object')

In [209]:
comprehensive_vault_stats['ilk'].unique()

array(['BAT-A', 'ETH-A', 'USDC-A', 'WBTC-A', 'USDC-B', 'TUSD-A', 'ZRX-A',
       'KNC-A', 'MANA-A', 'USDT-A', 'PAXUSD-A', 'LRC-A', 'COMP-A',
       'LINK-A', 'ETH-B', 'BAL-A', 'YFI-A', 'GUSD-A', 'UNI-A', 'RENBTC-A',
       'AAVE-A', 'UNIV2DAIETH-A', 'PSM-USDC-A', 'UNIV2USDCETH-A',
       'UNIV2WBTCETH-A', 'UNIV2DAIUSDC-A', 'UNIV2LINKETH-A',
       'UNIV2UNIETH-A', 'UNIV2AAVEETH-A', 'UNIV2WBTCDAI-A',
       'UNIV2DAIUSDT-A', 'ETH-C', 'RWA002-A', 'RWA001-A', 'RWA003-A',
       'RWA005-A', 'RWA006-A', 'RWA004-A', 'MATIC-A', 'PSM-PAX-A',
       'GUNIV3DAIUSDC1-A', 'WSTETH-A', 'WBTC-B', 'WBTC-C', 'PSM-GUSD-A',
       'GUNIV3DAIUSDC2-A', 'CRVV1ETHSTETH-A', 'WSTETH-B', 'RWA009-A',
       'RETH-A', 'RWA008-A', 'GNO-A', 'RWA013-A', 'RWA012-A'],
      dtype=object)

In [210]:
comprehensive_vault_stats['status'].head()

21751    Closed
21750    Closed
21748    Closed
21747    Closed
21746    Closed
Name: status, dtype: object

In [211]:
no_nan_vaults = comprehensive_vault_stats.copy()
no_nan_vaults = no_nan_vaults.fillna(0)

In [291]:
comprehensive_vault_stats.shape[0]

21499

In [305]:
comprehensive_vault_stats['market_price'].describe()

count          21,499.00
mean       47,931,037.04
std       344,766,479.83
min                 0.00
25%                 1.00
50%             1,380.00
75%            23,449.74
max     4,482,829,901.87
Name: market_price, dtype: float64

In [302]:
comprehensive_vault_stats.columns

Index(['day', 'cumulative_collateral', 'ilk', 'safety_price',
       'usd_safety_value', 'dai_ceiling', 'dai_floor',
       'safety_collateral_ratio', 'market_price', 'debt_balance',
       'liquidation_penalty', 'liquidation_ratio', 'daily_revenues',
       'annualized stability fee', 'annualized_revenues', 'dart',
       'total_ann_revenues', 'prev_dai_ceiling', 'status',
       'market_collateral_ratio', 'collateral_usd',
       'hypothetical_dai_ceiling'],
      dtype='object')

In [213]:
comprehensive_vault_stats[comprehensive_vault_stats['ilk']=='ETH-A']

,day,cumulative_collateral,ilk,safety_price,usd_safety_value,dai_ceiling,dai_floor,safety_collateral_ratio,market_price,debt_balance,...,daily_revenues,annualized stability fee,annualized_revenues,dart,total_ann_revenues,prev_dai_ceiling,status,market_collateral_ratio,collateral_usd,hypothetical_dai_ceiling
21750,2019-11-13 00:00:00+00:00,0.42,ETH-A,0.00,0.00,0.00,20.00,0.00,0.00,NaN,...,0.00,0.04,NaN,NaN,NaN,NaN,Closed,NaN,NaN,NaN
21747,2019-11-15 00:00:00+00:00,0.42,ETH-A,119.74,50.29,0.00,20.00,0.00,119.74,NaN,...,0.00,0.04,NaN,NaN,NaN,0.00,Closed,NaN,NaN,NaN
21745,2019-11-16 00:00:00+00:00,0.42,ETH-A,121.69,51.11,0.00,20.00,0.00,121.69,NaN,...,0.00,0.04,NaN,NaN,NaN,0.00,Closed,NaN,NaN,NaN
21743,2019-11-17 00:00:00+00:00,0.43,ETH-A,123.31,53.02,0.00,20.00,0.00,123.31,NaN,...,0.00,0.04,NaN,NaN,NaN,0.00,Closed,NaN,NaN,NaN
21742,2019-11-18 00:00:00+00:00,"26,313.39",ETH-A,118.39,"3,115,153.99","50,000,000.00",20.00,2.00,177.58,"1,558,617.22",...,37.68,0.04,"62,331.01","1,558,275.09","145,487.20",0.00,Open,3.00,"4,672,730.98","3,504,548.23"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37,2024-03-17 00:00:00+00:00,"444,063.07",ETH-A,"2,504.00","1,111,933,925.89","428,458,674.37","7,500.00",4.51,"3,630.80","246,751,192.26",...,"135,601.75",0.15,"37,629,556.50","246,751,192.26","209,313,743.80","428,458,674.37",Open,6.53,"1,612,304,192.54","1,168,920,539.59"
28,2024-03-18 00:00:00+00:00,"443,694.77",ETH-A,"2,439.83","1,082,537,747.09","428,458,674.37","7,500.00",4.38,"3,537.75","247,201,188.46",...,"77,882.14",0.15,"37,698,180.92","247,201,188.46","206,241,667.90","428,458,674.37",Open,6.35,"1,569,679,733.27","1,138,017,806.62"
18,2024-03-19 00:00:00+00:00,"442,822.50",ETH-A,"2,262.59","1,001,924,077.88","428,458,674.37","7,500.00",4.08,"3,280.75","245,449,240.77",...,"171,181.44",0.15,"37,431,008.90","245,449,240.77","199,379,356.93","428,458,674.37",Open,5.92,"1,452,789,912.92","1,053,272,686.87"
16,2024-03-20 00:00:00+00:00,"444,855.53",ETH-A,"2,402.78","1,068,888,133.11","428,458,674.37","7,500.00",4.35,"3,484.03","245,482,433.74",...,"136,276.39",0.15,"37,436,070.83","245,482,433.74","203,058,946.36","428,458,674.37",Open,6.31,"1,549,887,793.01","1,123,668,649.93"


In [214]:
no_nan_vaults[no_nan_vaults['ilk']=='WSTETH-A'].head()

,day,cumulative_collateral,ilk,safety_price,usd_safety_value,dai_ceiling,dai_floor,safety_collateral_ratio,market_price,debt_balance,...,daily_revenues,annualized stability fee,annualized_revenues,dart,total_ann_revenues,prev_dai_ceiling,status,market_collateral_ratio,collateral_usd,hypothetical_dai_ceiling
9079,2021-10-28 00:00:00+00:00,0.00,WSTETH-A,"2,697.62",0.00,"3,000,000.00","10,000.00",0.00,0.00,0.00,...,0.00,0.04,0.00,0.00,"76,729,520.44",0.00,Open,0.00,0.00,0.00
9075,2021-10-29 00:00:00+00:00,305.63,WSTETH-A,"2,857.24","873,269.68","3,000,000.00","10,000.00",1.38,0.00,"632,462.53",...,40.06,0.04,"25,298.50","632,462.53","80,139,053.59","3,000,000.00",Open,0.00,0.00,0.00
9062,2021-10-30 00:00:00+00:00,674.45,WSTETH-A,"2,811.58","1,896,273.33","4,078,604.29","10,000.00",1.70,0.00,"1,118,390.53",...,128.36,0.04,"44,735.62","1,118,390.53","81,886,606.49","3,000,000.00",Open,0.00,0.00,0.00
9056,2021-10-31 00:00:00+00:00,962.89,WSTETH-A,"2,730.51","2,629,187.21","4,353,719.21","10,000.00",1.92,0.00,"1,368,324.39",...,99.96,0.04,"54,732.98","1,368,324.39","85,368,386.53","4,078,604.29",Open,0.00,0.00,0.00
9043,2021-11-01 00:00:00+00:00,"3,417.46",WSTETH-A,"2,831.66","9,677,082.31","4,443,895.33","10,000.00",2.18,0.00,"4,442,038.79",...,264.64,0.04,"177,681.56","4,442,038.79","84,031,640.03","4,353,719.21",Open,0.00,0.00,0.00


In [215]:
top_10_ilks = top_10_vaults.index.tolist()

In [216]:
top_10_ilks

['ETH-A',
 'WBTC-A',
 'WSTETH-A',
 'ETH-C',
 'ETH-B',
 'WSTETH-B',
 'USDC-A',
 'RWA013-A',
 'WBTC-C',
 'RWA012-A']

In [217]:
topvaults = no_nan_vaults[no_nan_vaults['ilk'].isin(top_10_ilks)]

In [218]:
wbtc_a_vault = no_nan_vaults[no_nan_vaults['ilk']=='WBTC-A']
eth_a_vault = no_nan_vaults[no_nan_vaults['ilk']=='ETH-A']
wsteth_a_vault = no_nan_vaults[no_nan_vaults['ilk']=='WSTETH-A']
eth_c_vault = no_nan_vaults[no_nan_vaults['ilk']=='ETH-C']
eth_b_vault = no_nan_vaults[no_nan_vaults['ilk']=='ETH-B']

In [219]:
#We start with eth-a vault, which has long history and has generated most revenues 
eth_a_vault.head()

,day,cumulative_collateral,ilk,safety_price,usd_safety_value,dai_ceiling,dai_floor,safety_collateral_ratio,market_price,debt_balance,...,daily_revenues,annualized stability fee,annualized_revenues,dart,total_ann_revenues,prev_dai_ceiling,status,market_collateral_ratio,collateral_usd,hypothetical_dai_ceiling
21750,2019-11-13 00:00:00+00:00,0.42,ETH-A,0.00,0.00,0.00,20.00,0.00,0.00,0.00,...,0.00,0.04,0.00,0.00,0.00,0.00,Closed,0.00,0.00,0.00
21747,2019-11-15 00:00:00+00:00,0.42,ETH-A,119.74,50.29,0.00,20.00,0.00,119.74,0.00,...,0.00,0.04,0.00,0.00,0.00,0.00,Closed,0.00,0.00,0.00
21745,2019-11-16 00:00:00+00:00,0.42,ETH-A,121.69,51.11,0.00,20.00,0.00,121.69,0.00,...,0.00,0.04,0.00,0.00,0.00,0.00,Closed,0.00,0.00,0.00
21743,2019-11-17 00:00:00+00:00,0.43,ETH-A,123.31,53.02,0.00,20.00,0.00,123.31,0.00,...,0.00,0.04,0.00,0.00,0.00,0.00,Closed,0.00,0.00,0.00
21742,2019-11-18 00:00:00+00:00,"26,313.39",ETH-A,118.39,"3,115,153.99","50,000,000.00",20.00,2.00,177.58,"1,558,617.22",...,37.68,0.04,"62,331.01","1,558,275.09","145,487.20",0.00,Open,3.00,"4,672,730.98","3,504,548.23"


In [220]:
eth_a_vault.describe()

,cumulative_collateral,safety_price,usd_safety_value,dai_ceiling,dai_floor,safety_collateral_ratio,market_price,debt_balance,liquidation_penalty,liquidation_ratio,daily_revenues,annualized stability fee,annualized_revenues,dart,total_ann_revenues,prev_dai_ceiling,market_collateral_ratio,collateral_usd,hypothetical_dai_ceiling
count,"1,564.00","1,564.00","1,564.00","1,564.00","1,564.00","1,564.00","1,564.00","1,564.00","1,564.00","1,564.00","1,564.00","1,564.00","1,564.00","1,564.00","1,564.00","1,564.00","1,564.00","1,564.00","1,564.00"
mean,"1,458,734.70","1,180.31","1,674,815,093.22","854,497,895.35","7,344.54",2.66,"1,724.99","693,477,623.79",0.07,1.47,"70,427.29",0.03,"21,123,475.17","693,604,670.68","51,565,245.82","854,223,944.79",3.91,"2,461,159,737.81","1,808,848,479.57"
std,"867,527.31",785.15,"1,760,759,531.51","836,076,226.20","5,719.60",0.59,"1,141.89","788,907,642.18",0.16,0.08,"97,692.23",0.02,"27,873,569.54","789,103,240.00","53,855,258.00","836,286,080.97",0.84,"2,584,183,106.69","1,897,827,173.00"
min,0.42,0.00,0.00,0.00,20.00,0.00,0.00,0.00,-1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
25%,"517,750.16",396.93,"522,930,296.41","350,558,569.73",500.00,2.23,595.39,"179,038,619.21",0.00,1.45,"10,818.52",0.01,"3,031,349.40","179,038,619.21","12,664,699.17","350,558,569.73",3.30,"758,614,754.09","550,071,332.69"
50%,"1,733,410.48","1,171.43","673,651,523.57","421,276,896.85","7,500.00",2.56,"1,708.09","216,232,858.05",0.13,1.45,"26,452.76",0.02,"7,708,878.63","216,270,428.76","35,828,999.87","421,276,896.85",3.75,"992,240,131.57","726,560,471.73"
75%,"2,075,231.70","1,624.46","3,121,970,537.42","1,494,643,974.46","15,000.00",3.03,"2,372.86","1,340,082,473.97",0.13,1.50,"108,883.03",0.04,"33,206,894.43","1,340,082,473.97","73,813,092.27","1,494,643,974.46",4.42,"4,612,763,346.09","3,407,265,929.20"
max,"2,966,468.34","3,298.34","7,428,423,394.60","3,395,556,201.44","15,000.00",5.27,"4,782.60","3,077,919,054.99",0.13,1.50,"1,475,897.04",0.15,"142,209,506.83","3,077,919,054.99","351,700,458.61","3,395,556,201.44",7.63,"11,142,635,091.90","8,356,976,318.93"


In [221]:
wsteth_a_vault.describe()

,cumulative_collateral,safety_price,usd_safety_value,dai_ceiling,dai_floor,safety_collateral_ratio,market_price,debt_balance,liquidation_penalty,liquidation_ratio,daily_revenues,annualized stability fee,annualized_revenues,dart,total_ann_revenues,prev_dai_ceiling,market_collateral_ratio,collateral_usd,hypothetical_dai_ceiling
count,694.00,694.00,694.00,694.00,694.00,694.00,694.00,694.00,694.00,694.00,694.00,694.00,694.00,694.00,694.00,694.00,694.00,694.00,694.00
mean,"235,284.59","1,559.60","341,161,298.96","180,655,050.07","11,437.32",2.35,"2,379.47","148,401,531.70",0.06,1.55,"31,275.72",0.03,"6,331,998.71","148,397,826.28","61,180,282.34","180,304,451.06",3.66,"531,123,607.78","414,006,384.71"
std,"123,693.61",581.65,"205,675,977.89","104,785,968.65","3,672.67",0.42,918.21,"96,952,376.70",0.28,0.20,"47,118.74",0.02,"9,165,301.43","96,954,817.71","49,837,701.35","104,982,877.36",0.82,"310,175,433.29","234,498,518.48"
min,0.00,557.95,0.00,"3,000,000.00","7,500.00",0.00,0.00,0.00,-1.00,0.00,0.00,0.01,0.00,0.00,"10,595,628.80",0.00,0.00,0.00,0.00
25%,"131,492.62","1,142.57","148,729,635.81","87,260,831.15","7,500.00",2.06,"1,785.89","55,591,677.39",0.13,1.50,"7,794.98",0.02,"1,731,495.70","55,591,677.39","16,704,850.70","87,260,831.15",3.22,"237,967,417.30","190,373,933.84"
50%,"275,605.93","1,364.31","329,206,667.27","158,992,517.71","15,000.00",2.31,"2,135.15","139,576,792.35",0.13,1.60,"14,016.71",0.03,"2,473,128.77","139,576,792.35","60,664,028.43","158,992,517.71",3.57,"521,986,966.28","404,180,363.69"
75%,"336,262.00","1,928.03","459,540,170.88","244,315,338.57","15,000.00",2.53,"2,927.83","197,347,390.22",0.13,1.60,"38,821.51",0.04,"9,323,614.85","197,347,390.22","81,275,672.37","244,315,338.57",4.03,"718,101,258.75","571,317,388.08"
max,"607,527.12","3,122.84","920,944,499.06","500,000,000.00","15,000.00",3.65,"4,835.17","479,752,486.62",0.13,1.60,"630,005.97",0.16,"66,357,061.26","479,752,486.62","267,903,732.88","500,000,000.00",5.84,"1,381,416,748.59","1,070,656,199.35"


In [222]:
eth_a_vault.columns

Index(['day', 'cumulative_collateral', 'ilk', 'safety_price',
       'usd_safety_value', 'dai_ceiling', 'dai_floor',
       'safety_collateral_ratio', 'market_price', 'debt_balance',
       'liquidation_penalty', 'liquidation_ratio', 'daily_revenues',
       'annualized stability fee', 'annualized_revenues', 'dart',
       'total_ann_revenues', 'prev_dai_ceiling', 'status',
       'market_collateral_ratio', 'collateral_usd',
       'hypothetical_dai_ceiling'],
      dtype='object')

In [223]:
eth_a_vault.columns = [f'eth_a_vault_{col}' if col != 'period' and not col.startswith('day') else col for col in eth_a_vault.columns]

In [224]:
eth_a_vault.set_index('day', inplace=True)
eth_a_vault = eth_a_vault.drop(columns=['eth_a_vault_ilk'])

In [225]:
eth_a_vault.head()

,eth_a_vault_cumulative_collateral,eth_a_vault_safety_price,eth_a_vault_usd_safety_value,eth_a_vault_dai_ceiling,eth_a_vault_dai_floor,eth_a_vault_safety_collateral_ratio,eth_a_vault_market_price,eth_a_vault_debt_balance,eth_a_vault_liquidation_penalty,eth_a_vault_liquidation_ratio,eth_a_vault_daily_revenues,eth_a_vault_annualized stability fee,eth_a_vault_annualized_revenues,eth_a_vault_dart,eth_a_vault_total_ann_revenues,eth_a_vault_prev_dai_ceiling,eth_a_vault_status,eth_a_vault_market_collateral_ratio,eth_a_vault_collateral_usd,eth_a_vault_hypothetical_dai_ceiling
day,,,,,,,,,,,,,,,,,,,,
2019-11-13 00:00:00+00:00,0.42,0.00,0.00,0.00,20.00,0.00,0.00,0.00,0.00,0.00,0.00,0.04,0.00,0.00,0.00,0.00,Closed,0.00,0.00,0.00
2019-11-15 00:00:00+00:00,0.42,119.74,50.29,0.00,20.00,0.00,119.74,0.00,0.00,0.00,0.00,0.04,0.00,0.00,0.00,0.00,Closed,0.00,0.00,0.00
2019-11-16 00:00:00+00:00,0.42,121.69,51.11,0.00,20.00,0.00,121.69,0.00,0.00,0.00,0.00,0.04,0.00,0.00,0.00,0.00,Closed,0.00,0.00,0.00
2019-11-17 00:00:00+00:00,0.43,123.31,53.02,0.00,20.00,0.00,123.31,0.00,0.00,0.00,0.00,0.04,0.00,0.00,0.00,0.00,Closed,0.00,0.00,0.00
2019-11-18 00:00:00+00:00,"26,313.39",118.39,"3,115,153.99","50,000,000.00",20.00,2.00,177.58,"1,558,617.22",0.00,1.50,37.68,0.04,"62,331.01","1,558,275.09","145,487.20",0.00,Open,3.00,"4,672,730.98","3,504,548.23"


In [226]:
wbtc_a_vault.columns = [f'wbtc_vault_{col}' if col != 'period' and not col.startswith('day') else col for col in wbtc_a_vault.columns]

In [227]:
wbtc_a_vault.tail()

,day,wbtc_vault_cumulative_collateral,wbtc_vault_ilk,wbtc_vault_safety_price,wbtc_vault_usd_safety_value,wbtc_vault_dai_ceiling,wbtc_vault_dai_floor,wbtc_vault_safety_collateral_ratio,wbtc_vault_market_price,wbtc_vault_debt_balance,...,wbtc_vault_daily_revenues,wbtc_vault_annualized stability fee,wbtc_vault_annualized_revenues,wbtc_vault_dart,wbtc_vault_total_ann_revenues,wbtc_vault_prev_dai_ceiling,wbtc_vault_status,wbtc_vault_market_collateral_ratio,wbtc_vault_collateral_usd,wbtc_vault_hypothetical_dai_ceiling
36,2024-03-17 00:00:00+00:00,"3,802.16",WBTC-A,"47,221.06","179,542,195.20","74,121,006.59","7,500.00",2.85,"68,470.54","63,025,368.51",...,"106,434.04",0.17,"10,556,749.35","63,025,368.51","209,313,743.80","73,895,238.26",Open,4.13,"260,336,183.04","188,743,732.71"
27,2024-03-18 00:00:00+00:00,"3,800.90",WBTC-A,"46,586.76","177,071,699.86","76,495,946.51","7,500.00",2.79,"67,550.80","63,432,428.49",...,"55,892.88",0.17,"10,624,931.90","63,432,428.49","206,241,667.90","74,121,006.59",Open,4.05,"256,753,964.80","186,146,624.48"
19,2024-03-19 00:00:00+00:00,"3,801.52",WBTC-A,"44,054.59","167,474,331.52","76,495,946.51","7,500.00",2.65,"63,879.16","63,104,799.10",...,"78,303.14",0.17,"10,570,053.98","63,104,799.10","199,379,356.93","76,495,946.51",Open,3.85,"242,837,780.71","176,057,391.01"
10,2024-03-20 00:00:00+00:00,"3,796.14",WBTC-A,"45,191.77","171,554,182.95","76,495,946.51","7,500.00",2.72,"65,528.06","63,116,858.78",...,"51,171.83",0.17,"10,572,073.97","63,116,858.78","203,058,946.36","76,495,946.51",Open,3.94,"248,753,565.27","180,346,334.82"
4,2024-03-21 00:00:00+00:00,"3,792.99",WBTC-A,"44,840.72","170,080,439.25","76,495,946.51","7,500.00",2.70,"65,019.05","63,091,075.47",...,"78,576.81",0.17,"10,567,608.94","63,090,201.85","176,643,660.32","76,495,946.51",Open,3.91,"246,616,636.92","178,797,061.76"


In [228]:
dai_maturity_df_reindexed.head()

,day,dai_maturity_outflow_1-block,dai_maturity_outflow_1-day,dai_maturity_outflow_1-month,dai_maturity_outflow_1-week,dai_maturity_outflow_1-year,dai_maturity_outflow_3-months,dai_maturity_outflow_dai_only_1-block,dai_maturity_outflow_dai_only_1-day,dai_maturity_outflow_dai_only_1-month,dai_maturity_outflow_dai_only_1-week,dai_maturity_outflow_dai_only_1-year,dai_maturity_outflow_dai_only_3-months,dai_maturity_outflow_surplus_buffer_1-block,dai_maturity_outflow_surplus_buffer_1-day,dai_maturity_outflow_surplus_buffer_1-month,dai_maturity_outflow_surplus_buffer_1-week,dai_maturity_outflow_surplus_buffer_1-year,dai_maturity_outflow_surplus_buffer_3-months
0,2019-11-13 00:00:00+00:00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,2019-11-14 00:00:00+00:00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,2019-11-15 00:00:00+00:00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,2019-11-16 00:00:00+00:00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,2019-11-17 00:00:00+00:00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [229]:
eth_a_vault = eth_a_vault.merge(dai_maturity_df_reindexed, on=['day'], how='inner')

In [230]:
stable_coin_ratios.head()

,stablecoins_ratio,usdc_ratio
day,,
2024-03-21 00:00:00+00:00,0.11,0.11
2024-03-20 00:00:00+00:00,0.10,0.10
2024-03-19 00:00:00+00:00,0.09,0.09
2024-03-18 00:00:00+00:00,0.08,0.08
2024-03-17 00:00:00+00:00,0.13,0.13


In [231]:
#eth_a_vault = eth_a_vault.merge(stable_coin_ratios, on=['day'], how='inner')

In [232]:
eth_a_vault.head()

,day,eth_a_vault_cumulative_collateral,eth_a_vault_safety_price,eth_a_vault_usd_safety_value,eth_a_vault_dai_ceiling,eth_a_vault_dai_floor,eth_a_vault_safety_collateral_ratio,eth_a_vault_market_price,eth_a_vault_debt_balance,eth_a_vault_liquidation_penalty,...,dai_maturity_outflow_dai_only_1-month,dai_maturity_outflow_dai_only_1-week,dai_maturity_outflow_dai_only_1-year,dai_maturity_outflow_dai_only_3-months,dai_maturity_outflow_surplus_buffer_1-block,dai_maturity_outflow_surplus_buffer_1-day,dai_maturity_outflow_surplus_buffer_1-month,dai_maturity_outflow_surplus_buffer_1-week,dai_maturity_outflow_surplus_buffer_1-year,dai_maturity_outflow_surplus_buffer_3-months
0,2019-11-13 00:00:00+00:00,0.42,0.00,0.00,0.00,20.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,2019-11-15 00:00:00+00:00,0.42,119.74,50.29,0.00,20.00,0.00,119.74,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,2019-11-16 00:00:00+00:00,0.42,121.69,51.11,0.00,20.00,0.00,121.69,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,2019-11-17 00:00:00+00:00,0.43,123.31,53.02,0.00,20.00,0.00,123.31,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,2019-11-18 00:00:00+00:00,"26,313.39",118.39,"3,115,153.99","50,000,000.00",20.00,2.00,177.58,"1,558,617.22",0.00,...,"44,042.45","201,006.77","1,173,199.67","64,980.67",0.00,0.00,0.00,0.00,90.79,0.00


In [233]:
start_date = eth_a_vault['day'].min()
end_date = eth_a_vault['day'].max()
date_range = pd.date_range(start=start_date, end=end_date)

psm_full_range_df = pd.DataFrame(index=date_range)



In [234]:
psm_full_range_df.head()

""
2019-11-13 00:00:00+00:00
2019-11-14 00:00:00+00:00
2019-11-15 00:00:00+00:00
2019-11-16 00:00:00+00:00
2019-11-17 00:00:00+00:00


In [235]:


psm_columns = ['psm_change', 'psm_change_excl_rwa', 'psm_change_excl_rwa_30d_avg', 'psm_change_excl_rwa_7d_avg', 'psm_fees', 'psm_inflow', 'psm_inflow_exl_rwa', 'psm_lifetime_fees', 'psm_lifetime_turnover', 'psm_outflow', 'psm_balance', 'psm_turnover']
for column in psm_columns:
    psm_full_range_df[column] = psm_stats_csv[column]

# Fill missing values with zeros
psm_full_range_df.fillna(0, inplace=True)

In [236]:
eth_a_vault.describe()

,eth_a_vault_cumulative_collateral,eth_a_vault_safety_price,eth_a_vault_usd_safety_value,eth_a_vault_dai_ceiling,eth_a_vault_dai_floor,eth_a_vault_safety_collateral_ratio,eth_a_vault_market_price,eth_a_vault_debt_balance,eth_a_vault_liquidation_penalty,eth_a_vault_liquidation_ratio,...,dai_maturity_outflow_dai_only_1-month,dai_maturity_outflow_dai_only_1-week,dai_maturity_outflow_dai_only_1-year,dai_maturity_outflow_dai_only_3-months,dai_maturity_outflow_surplus_buffer_1-block,dai_maturity_outflow_surplus_buffer_1-day,dai_maturity_outflow_surplus_buffer_1-month,dai_maturity_outflow_surplus_buffer_1-week,dai_maturity_outflow_surplus_buffer_1-year,dai_maturity_outflow_surplus_buffer_3-months
count,"1,563.00","1,563.00","1,563.00","1,563.00","1,563.00","1,563.00","1,563.00","1,563.00","1,563.00","1,563.00",...,"1,563.00","1,563.00","1,563.00","1,563.00","1,563.00","1,563.00","1,563.00","1,563.00","1,563.00","1,563.00"
mean,"1,459,396.78","1,179.55","1,675,246,787.68","854,770,473.22","7,344.44",2.66,"1,723.91","693,783,096.07",0.07,1.47,...,"121,829,547.87","534,720,838.07","2,643,717,876.30","179,748,513.26",0.00,0.00,0.00,0.00,"43,165,723.38",0.00
std,"867,409.61",784.84,"1,761,240,268.84","836,274,295.12","5,721.43",0.59,"1,141.45","789,067,601.73",0.16,0.08,...,"86,900,656.56","372,490,872.45","1,759,401,866.73","128,214,083.44",0.00,0.00,0.00,0.00,"30,383,977.10",0.00
min,0.42,0.00,0.00,0.00,20.00,0.00,0.00,0.00,-1.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"-5,674,219.65",0.00
25%,"517,894.21",396.45,"522,790,141.46","350,558,569.73",500.00,2.23,594.67,"179,036,518.30",0.00,1.45,...,"42,015,130.91","170,882,210.19","570,286,315.21","61,989,537.40",0.00,0.00,0.00,0.00,"4,001,819.24",0.00
50%,"1,734,175.26","1,171.10","673,540,872.03","421,276,896.85","7,500.00",2.56,"1,707.98","216,249,923.32",0.13,1.45,...,"118,349,364.98","539,666,807.68","3,212,265,572.50","174,613,817.18",0.00,0.00,0.00,0.00,"53,218,504.18",0.00
75%,"2,075,622.02","1,622.45","3,122,870,291.31","1,496,694,216.25","15,000.00",3.03,"2,370.73","1,340,283,261.88",0.13,1.50,...,"168,159,893.31","742,722,843.39","3,705,032,007.03","248,104,760.62",0.00,0.00,0.00,0.00,"69,846,462.07",0.00
max,"2,966,468.34","3,298.34","7,428,423,394.60","3,395,556,201.44","15,000.00",5.27,"4,782.60","3,077,919,054.99",0.13,1.50,...,"314,069,539.85","1,342,922,289.47","5,952,992,110.30","463,381,288.30",0.00,0.00,0.00,0.00,"83,550,327.25",0.00


In [237]:
eth_a_vault.head()

,day,eth_a_vault_cumulative_collateral,eth_a_vault_safety_price,eth_a_vault_usd_safety_value,eth_a_vault_dai_ceiling,eth_a_vault_dai_floor,eth_a_vault_safety_collateral_ratio,eth_a_vault_market_price,eth_a_vault_debt_balance,eth_a_vault_liquidation_penalty,...,dai_maturity_outflow_dai_only_1-month,dai_maturity_outflow_dai_only_1-week,dai_maturity_outflow_dai_only_1-year,dai_maturity_outflow_dai_only_3-months,dai_maturity_outflow_surplus_buffer_1-block,dai_maturity_outflow_surplus_buffer_1-day,dai_maturity_outflow_surplus_buffer_1-month,dai_maturity_outflow_surplus_buffer_1-week,dai_maturity_outflow_surplus_buffer_1-year,dai_maturity_outflow_surplus_buffer_3-months
0,2019-11-13 00:00:00+00:00,0.42,0.00,0.00,0.00,20.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,2019-11-15 00:00:00+00:00,0.42,119.74,50.29,0.00,20.00,0.00,119.74,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,2019-11-16 00:00:00+00:00,0.42,121.69,51.11,0.00,20.00,0.00,121.69,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,2019-11-17 00:00:00+00:00,0.43,123.31,53.02,0.00,20.00,0.00,123.31,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,2019-11-18 00:00:00+00:00,"26,313.39",118.39,"3,115,153.99","50,000,000.00",20.00,2.00,177.58,"1,558,617.22",0.00,...,"44,042.45","201,006.77","1,173,199.67","64,980.67",0.00,0.00,0.00,0.00,90.79,0.00


In [238]:
psm_full_range_df = psm_full_range_df.rename_axis('day')

In [239]:
psm_full_range_df.duplicated().sum()

424

In [240]:
eth_a_vault= eth_a_vault.merge(psm_full_range_df, on=['day'], how='inner')

In [241]:
eth_a_vault[['day','psm_change']].tail()

,day,psm_change
1558,2024-03-16 00:00:00+00:00,"6,819,615.50"
1559,2024-03-17 00:00:00+00:00,"64,591,276.21"
1560,2024-03-18 00:00:00+00:00,"-232,459,681.52"
1561,2024-03-19 00:00:00+00:00,"4,430,852.16"
1562,2024-03-20 00:00:00+00:00,"47,004,882.05"


In [242]:
where_is_dai_csv_table_full = pd.DataFrame(index=date_range)


where_is_dai_csv_table_columns = where_is_dai_csv_table.columns
for column in where_is_dai_csv_table_columns:
    where_is_dai_csv_table_full[column] = where_is_dai_csv_table[column]

# Fill missing values with zeros
where_is_dai_csv_table_full.fillna(0, inplace=True)

In [243]:
where_is_dai_csv_table_full = where_is_dai_csv_table_full.rename_axis('day')

In [244]:
where_is_dai_csv_table_full.head()

,where_is_dai_Bridge,where_is_dai_CeFi,where_is_dai_Dai Savings,where_is_dai_Dex,where_is_dai_EOA,where_is_dai_Lending,where_is_dai_Other,where_is_dai_Treasury
day,,,,,,,,
2019-11-13 00:00:00+00:00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2019-11-14 00:00:00+00:00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2019-11-15 00:00:00+00:00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2019-11-16 00:00:00+00:00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2019-11-17 00:00:00+00:00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [245]:
eth_a_vault= eth_a_vault.merge(where_is_dai_csv_table_full, on=['day'], how='inner')

In [246]:
eth_a_vault.head()

,day,eth_a_vault_cumulative_collateral,eth_a_vault_safety_price,eth_a_vault_usd_safety_value,eth_a_vault_dai_ceiling,eth_a_vault_dai_floor,eth_a_vault_safety_collateral_ratio,eth_a_vault_market_price,eth_a_vault_debt_balance,eth_a_vault_liquidation_penalty,...,psm_balance,psm_turnover,where_is_dai_Bridge,where_is_dai_CeFi,where_is_dai_Dai Savings,where_is_dai_Dex,where_is_dai_EOA,where_is_dai_Lending,where_is_dai_Other,where_is_dai_Treasury
0,2019-11-13 00:00:00+00:00,0.42,0.00,0.00,0.00,20.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,2019-11-15 00:00:00+00:00,0.42,119.74,50.29,0.00,20.00,0.00,119.74,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,2019-11-16 00:00:00+00:00,0.42,121.69,51.11,0.00,20.00,0.00,121.69,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,2019-11-17 00:00:00+00:00,0.43,123.31,53.02,0.00,20.00,0.00,123.31,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,2019-11-18 00:00:00+00:00,"26,313.39",118.39,"3,115,153.99","50,000,000.00",20.00,2.00,177.58,"1,558,617.22",0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [247]:
daily_surplus_buffer_csv = daily_surplus_buffer_csv.fillna(0)

In [248]:
eth_a_vault= eth_a_vault.merge(daily_surplus_buffer_csv, on=['day'], how='inner')

In [249]:
eth_a_vault.head()

,day,eth_a_vault_cumulative_collateral,eth_a_vault_safety_price,eth_a_vault_usd_safety_value,eth_a_vault_dai_ceiling,eth_a_vault_dai_floor,eth_a_vault_safety_collateral_ratio,eth_a_vault_market_price,eth_a_vault_debt_balance,eth_a_vault_liquidation_penalty,...,where_is_dai_CeFi,where_is_dai_Dai Savings,where_is_dai_Dex,where_is_dai_EOA,where_is_dai_Lending,where_is_dai_Other,where_is_dai_Treasury,daily_surplus_buffer_delta_30d,daily_surplus_buffer_delta_90d,daily_surplus_buffer
0,2019-11-13 00:00:00+00:00,0.42,0.00,0.00,0.00,20.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,2019-11-15 00:00:00+00:00,0.42,119.74,50.29,0.00,20.00,0.00,119.74,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,2019-11-16 00:00:00+00:00,0.42,121.69,51.11,0.00,20.00,0.00,121.69,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,2019-11-17 00:00:00+00:00,0.43,123.31,53.02,0.00,20.00,0.00,123.31,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,2019-11-18 00:00:00+00:00,"26,313.39",118.39,"3,115,153.99","50,000,000.00",20.00,2.00,177.58,"1,558,617.22",0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,90.78


In [250]:
dsr_df.fillna(0, inplace=True)

In [251]:
dsr_df.head()

,dsr_inflow,dsr_interest,dsr_outflow,dsr_balance,dsr_rate,dai_circulating,dai_total_balance,dai_percent_in_dsr,dai_percent_out_dsr
day,,,,,,,,,
2024-03-27,"5,288,164.43","146,319.97",0.00,"1,545,849,069.62",0.15,"3,198,754,360.76","4,744,603,430.38",0.33,0.67
2024-03-26,"24,182,507.40","570,070.21",0.00,"1,510,137,336.77",0.15,"3,243,870,446.94","4,754,007,783.71",0.32,0.68
2024-03-25,0.00,"591,988.67","79,508,005.52","1,485,384,759.15",0.15,"3,238,774,686.95","4,724,159,446.11",0.31,0.69
2024-03-24,"32,601,790.65","589,190.34",0.00,"1,564,300,776.00",0.15,"3,258,196,717.18","4,822,497,493.18",0.32,0.68
2024-03-23,"42,441,802.19","577,016.24",0.00,"1,531,109,795.00",0.15,"3,245,174,169.16","4,776,283,964.16",0.32,0.68


In [253]:
dsr_df.reset_index(inplace=True)
eth_a_vault.reset_index(inplace=True)
# Remove timezone information from both 'day' columns
eth_a_vault['day'] = eth_a_vault['day'].dt.tz_localize(None)
dsr_df['day'] = dsr_df['day'].dt.tz_localize(None)

In [254]:
dsr_df_full = pd.DataFrame(index=date_range)
dsr_df_full = dsr_df_full.rename_axis('day')

dsr_df_full.reset_index(inplace=True)
dsr_df_full['day'] = dsr_df_full['day'].dt.tz_localize(None)


In [255]:
dsr_df_full = dsr_df_full.merge(dsr_df, on=['day'], how='outer')

In [256]:
dsr_df_full.fillna(0, inplace=True)

In [257]:
dsr_df_full

,day,dsr_inflow,dsr_interest,dsr_outflow,dsr_balance,dsr_rate,dai_circulating,dai_total_balance,dai_percent_in_dsr,dai_percent_out_dsr
0,2019-11-13,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,2019-11-14,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,2019-11-15,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,2019-11-16,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,2019-11-17,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...
1592,2024-03-25,0.00,"591,988.67","79,508,005.52","1,485,384,759.15",0.15,"3,238,774,686.95","4,724,159,446.11",0.31,0.69
1593,2024-03-24,"32,601,790.65","589,190.34",0.00,"1,564,300,776.00",0.15,"3,258,196,717.18","4,822,497,493.18",0.32,0.68
1594,2024-03-23,"42,441,802.19","577,016.24",0.00,"1,531,109,795.00",0.15,"3,245,174,169.16","4,776,283,964.16",0.32,0.68
1595,2024-03-22,"80,719,471.60","542,875.90",0.00,"1,488,090,976.57",0.15,"3,217,359,661.11","4,705,450,637.68",0.32,0.68


In [258]:
dsr_df_full = dsr_df_full.set_index('day')

In [260]:
#dsr_df_full = dsr_df_full.drop(columns=['level_0','index'])

In [261]:
dsr_df_full.head()

,dsr_inflow,dsr_interest,dsr_outflow,dsr_balance,dsr_rate,dai_circulating,dai_total_balance,dai_percent_in_dsr,dai_percent_out_dsr
day,,,,,,,,,
2019-11-13,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2019-11-14,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2019-11-15,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2019-11-16,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2019-11-17,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [267]:
eth_a_vault= eth_a_vault.merge(dsr_df_full, on=['day'], how='inner')

In [270]:
eth_a_vault = eth_a_vault.drop(columns=['index'])

In [271]:
eth_a_vault.head()

,day,eth_a_vault_cumulative_collateral,eth_a_vault_safety_price,eth_a_vault_usd_safety_value,eth_a_vault_dai_ceiling,eth_a_vault_dai_floor,eth_a_vault_safety_collateral_ratio,eth_a_vault_market_price,eth_a_vault_debt_balance,eth_a_vault_liquidation_penalty,...,dai_percent_out_dsr_x,dsr_inflow_y,dsr_interest_y,dsr_outflow_y,dsr_balance_y,dsr_rate_y,dai_circulating_y,dai_total_balance_y,dai_percent_in_dsr_y,dai_percent_out_dsr_y
0,2019-11-13,0.42,0.00,0.00,0.00,20.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,2019-11-15,0.42,119.74,50.29,0.00,20.00,0.00,119.74,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,2019-11-16,0.42,121.69,51.11,0.00,20.00,0.00,121.69,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,2019-11-17,0.43,123.31,53.02,0.00,20.00,0.00,123.31,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,2019-11-18,"26,313.39",118.39,"3,115,153.99","50,000,000.00",20.00,2.00,177.58,"1,558,617.22",0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [272]:
pivoted_balance_sheet = pivoted_balance_sheet.rename_axis('day')

In [273]:
pivoted_income_statement.tail()

item,period,1 - PnL,1.1 - Lending Revenues,1.2 - Liquidations Revenues,1.3 - Trading Revenues,1.4 - Lending Expenses,1.5 - Liquidations Expenses,1.6 - Workforce Expenses,1.9 - Net Income,2 - Assets,...,debt_to_equity_Lag_11,debt_ratio_Lag_11,Total_Revenues_Lag_12,Total_Expenses_Lag_12,Net_Income_Lag_12,profit_margin_Lag_12,ROA_Lag_12,ROE_Lag_12,debt_to_equity_Lag_12,debt_ratio_Lag_12
48,2023-11-01,0.00,"29,666,040.79",455.03,0.00,"-6,376,789.60",0.00,"-2,974,302.25","20,315,403.96",0.00,...,67.65,0.99,"1,169,462.77","-1,941,130.27","-771,667.50",-0.00,-0.00,-0.01,68.99,0.99
49,2023-12-01,0.00,"14,270,261.48",0.00,0.00,"-6,484,200.84",0.00,"-2,658,930.56","5,127,130.08",0.00,...,69.23,0.99,"1,364,508.64","-1,849,641.34","-485,132.70",-0.00,-0.00,-0.01,67.65,0.99
50,2024-01-01,0.00,"20,751,255.62","27,376.30",0.00,"-5,943,924.23",0.00,"-5,669,555.33","9,165,152.35",0.00,...,69.70,0.99,"1,540,089.06","-2,003,227.02","-463,137.96",-0.00,-0.00,-0.01,69.23,0.99
51,2024-02-01,0.00,"29,691,662.12",0.00,0.00,"-4,567,709.04",0.00,"-1,581,115.17","23,542,837.91",0.00,...,72.85,0.99,"1,750,712.42","-2,422,673.84","-671,961.42",-0.00,-0.00,-0.01,69.70,0.99
52,2024-03-01,0.00,"18,130,771.84","4,623.88",0.00,"-7,076,544.94",0.00,"-3,857,879.46","7,200,971.32",0.00,...,68.99,0.99,"2,286,044.04","-4,456,169.85","-2,170,125.82",-0.00,-0.00,-0.03,72.85,0.99


In [274]:
pivoted_balance_sheet.reset_index(inplace=True)
pivoted_balance_sheet['day'] = pivoted_balance_sheet['day'].dt.tz_localize(None)

In [275]:
pivoted_balance_sheet = pivoted_balance_sheet.set_index('day')

In [276]:
pivoted_balance_sheet = pivoted_balance_sheet.fillna(0)

In [277]:
pivoted_balance_sheet.head()

,b_s_Crypto-Loans,b_s_DAI,b_s_DSR,b_s_Equity,b_s_Others assets,b_s_Real-World Assets,b_s_Stablecoins,b_s_Crypto-Loans_pct_chg,b_s_DAI_pct_chg,b_s_DSR_pct_chg,...,b_s_DSR_rolling_avg_pct_chg,b_s_DSR_volatility_pct_chg,b_s_Equity_rolling_avg_pct_chg,b_s_Equity_volatility_pct_chg,b_s_Others_assets_rolling_avg_pct_chg,b_s_Others_assets_volatility_pct_chg,b_s_Real-World_Assets_rolling_avg_pct_chg,b_s_Real-World_Assets_volatility_pct_chg,b_s_Stablecoins_rolling_avg_pct_chg,b_s_Stablecoins_volatility_pct_chg
day,,,,,,,,,,,,,,,,,,,,,
2020-07-01,"144,805,383.35","-111,909,502.58","-32,695,722.07","-200,158.70",0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2020-07-02,"149,825,080.52","-120,793,695.43","-28,829,536.58","-201,848.51",0.00,0.00,0.00,0.03,0.08,-0.12,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2020-07-03,"161,260,645.24","-112,830,855.41","-48,225,765.52","-204,024.31",0.00,0.00,0.00,0.08,-0.07,0.67,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2020-07-04,"169,866,432.50","-114,652,320.54","-55,007,677.90","-206,434.05",0.00,0.00,0.00,0.05,0.02,0.14,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2020-07-05,"173,312,164.26","-116,283,121.58","-56,819,607.23","-209,435.45",0.00,0.00,0.00,0.02,0.01,0.03,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [278]:
eth_a_vault= eth_a_vault.merge(pivoted_balance_sheet, on=['day'], how='left') 

In [279]:
eth_a_vault = eth_a_vault.fillna(0)

In [280]:
eth_a_vault.shape[0]

1563

In [281]:
print(list(eth_a_vault.columns))

['day', 'eth_a_vault_cumulative_collateral', 'eth_a_vault_safety_price', 'eth_a_vault_usd_safety_value', 'eth_a_vault_dai_ceiling', 'eth_a_vault_dai_floor', 'eth_a_vault_safety_collateral_ratio', 'eth_a_vault_market_price', 'eth_a_vault_debt_balance', 'eth_a_vault_liquidation_penalty', 'eth_a_vault_liquidation_ratio', 'eth_a_vault_daily_revenues', 'eth_a_vault_annualized stability fee', 'eth_a_vault_annualized_revenues', 'eth_a_vault_dart', 'eth_a_vault_total_ann_revenues', 'eth_a_vault_prev_dai_ceiling', 'eth_a_vault_status', 'eth_a_vault_market_collateral_ratio', 'eth_a_vault_collateral_usd', 'eth_a_vault_hypothetical_dai_ceiling', 'dai_maturity_outflow_1-block', 'dai_maturity_outflow_1-day', 'dai_maturity_outflow_1-month', 'dai_maturity_outflow_1-week', 'dai_maturity_outflow_1-year', 'dai_maturity_outflow_3-months', 'dai_maturity_outflow_dai_only_1-block', 'dai_maturity_outflow_dai_only_1-day', 'dai_maturity_outflow_dai_only_1-month', 'dai_maturity_outflow_dai_only_1-week', 'dai_mat

In [304]:
#eth_a_vault.set_index('day')


,index,eth_a_vault_cumulative_collateral,eth_a_vault_safety_price,eth_a_vault_usd_safety_value,eth_a_vault_dai_ceiling,eth_a_vault_dai_floor,eth_a_vault_safety_collateral_ratio,eth_a_vault_market_price,eth_a_vault_debt_balance,eth_a_vault_liquidation_penalty,...,b_s_DSR_rolling_avg_pct_chg,b_s_DSR_volatility_pct_chg,b_s_Equity_rolling_avg_pct_chg,b_s_Equity_volatility_pct_chg,b_s_Others_assets_rolling_avg_pct_chg,b_s_Others_assets_volatility_pct_chg,b_s_Real-World_Assets_rolling_avg_pct_chg,b_s_Real-World_Assets_volatility_pct_chg,b_s_Stablecoins_rolling_avg_pct_chg,b_s_Stablecoins_volatility_pct_chg
day,,,,,,,,,,,,,,,,,,,,,
2022-08-25,604,"499,739.09","1,178.62","589,002,831.14","350,558,569.73","15,000.00",3.85,"1,709.00","153,143,002.24",0.13,...,-0.00,0.00,0.00,0.01,0.00,0.00,0.03,0.13,-0.00,0.01
2022-08-26,605,"484,297.17","1,066.69","516,596,535.74","350,558,569.73","15,000.00",3.47,"1,546.71","148,702,308.75",0.13,...,-0.00,0.00,-0.00,0.00,0.00,0.00,0.03,0.13,-0.00,0.01
2022-08-27,606,"483,633.77","1,016.90","491,805,516.38","350,558,569.73","15,000.00",3.33,"1,474.50","147,587,388.30",0.13,...,-0.00,0.00,-0.00,0.00,0.00,0.00,0.03,0.13,-0.00,0.01
2022-08-28,607,"483,430.25","1,024.96","495,496,001.44","350,558,569.73","15,000.00",3.36,"1,486.19","147,418,914.05",0.13,...,-0.00,0.00,-0.00,0.00,0.00,0.00,0.03,0.13,-0.00,0.01
2022-08-29,608,"482,878.78","1,065.10","514,315,856.28","350,558,569.73","15,000.00",3.35,"1,544.40","153,594,292.97",0.13,...,-0.00,0.00,-0.00,0.00,0.00,0.00,0.04,0.14,-0.00,0.01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-03-16,1147,"444,596.44","2,452.41","1,090,334,438.32","428,458,674.37","7,500.00",4.40,"3,556.00","247,615,665.19",0.13,...,0.01,0.05,0.01,0.04,0.00,0.00,-0.02,0.03,0.03,0.17
2024-03-17,1148,"444,063.07","2,504.00","1,111,933,925.89","428,458,674.37","7,500.00",4.51,"3,630.80","246,751,192.26",0.13,...,0.01,0.05,0.01,0.04,0.00,0.00,-0.02,0.03,0.03,0.17
2024-03-18,1149,"443,694.77","2,439.83","1,082,537,747.09","428,458,674.37","7,500.00",4.38,"3,537.75","247,201,188.46",0.13,...,0.01,0.05,0.01,0.04,0.00,0.00,-0.02,0.03,0.03,0.17


### Now for CoinGecko Crypto Market Data

In [221]:
#lets get price feeds for accepted collateral types

ir_csv['ilk'].unique()

array(['ETH-A', 'ETH-B', 'ETH-C', 'WBTC-A', 'WBTC-B', 'WBTC-C',
       'WSTETH-A', 'WSTETH-B', 'RWA002-A', 'RWA013-A', 'DIRECT-SPARK-DAI',
       'RWA014-A', 'RWA005-A', 'RWA012-A', 'RWA015-A', 'RWA007-A',
       'RETH-A', 'RWA003-A', 'GUNIV3DAIUSDC2-A', 'CRVV1ETHSTETH-A',
       'USDC-B', 'LINK-A', 'MATIC-A', 'UNIV2USDCETH-A', 'GNO-A',
       'UNIV2DAIUSDC-A', 'YFI-A', 'RWA004-A', 'GUNIV3DAIUSDC1-A',
       'GUSD-A', 'PAXUSD-A', 'USDC-A', 'DIRECT-AAVEV2-DAI',
       'DIRECT-COMPV2-DAI', 'RWA008-A', 'RENBTC-A', 'MANA-A', 'RWA009-A',
       'RWA001-A', 'UNI-A', 'UNIV2DAIETH-A', 'UNIV2WBTCETH-A',
       'UNIV2WBTCDAI-A', 'RWA-001', 'UNIV2UNIETH-A', 'TUSD-A', 'USDP-A',
       'BAT-A', 'BAL-A', 'ZRX-A', 'COMP-A', 'PSM-GUSD-A', 'AAVE-A',
       'UNIV2LINKETH-A', 'KNC-A', 'LRC-A', 'PSM-USDC-A', 'UNIV2AAVEETH-A',
       'UNIV2DAIUSDT-A', 'UNIV2ETHUSDT-A', 'USDT-A', 'PSM-PAX-A',
       'RWA006-A', 'PAX-A', nan, 'USDC', 'SAI'], dtype=object)

In [227]:
#need to use yfinance instead, coingecko clocked to 1 year historical


eth_historical_api = "https://api.coingecko.com/api/v3/coins/ethereum/market_chart"
wsteth_historical_api = "https://api.coingecko.com/api/v3/coins/wrapped-steth/market_chart"
wbtc_historical_api = "https://api.coingecko.com/api/v3/coins/wrapped-bitcoin/market_chart"

In [608]:
eth_history, eth_historical_mk, eth_vol = fetch_historical_data(eth_historical_api, api_key_cg)
wsteth_history, wsteth_historical_mk, wsteth_vol = fetch_historical_data(wsteth_historical_api, api_key_cg)
wbtc_history, wbtc_historical_mk, wbtc_vol = fetch_historical_data(wbtc_historical_api, api_key_cg)

Failed to retrieve data: 401
Failed to retrieve data: 401
Failed to retrieve data: 401


In [229]:
wbtc_vol

""


In [230]:
wsteth_history

""


In [231]:
wbtc_history

""


In [232]:
# DAI Price feed and vol

dai_historical_api = "https://api.coingecko.com/api/v3/coins/dai/market_chart"

dai_history, dai_historical_mk, dai_vol = fetch_historical_data(dai_historical_api, api_key_cg)

Failed to retrieve data: 401


In [233]:
dai_vol

""
